In [1]:
!pip install lazypredict
!pip install openpyxl

  Using cached openpyxl-3.1.2-py2.py3-none-any.whl (249 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


In [2]:
import lazypredict
import pandas as pd
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score
import numpy as np

In [2]:
df = pd.read_csv("datasets/edited/mfrc_with_classes.csv")
del df['positive']
del df['negative']
del df['neutral']
df

,care.virtue,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
4,0.00,0.00,0.00,0.00,0.00,0.00,0.33,0.67,0.00,0.00,Morally Positive
...,...,...,...,...,...,...,...,...,...,...,...
17881,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
17882,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
17883,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,Morally Negative
17884,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [2]:
df = pd.read_csv("datasets/edited/davidson_with_classes.csv")
del df['positive']
del df['negative']
del df['neutral']
df

,care.virtue,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,1.00,Partially Negative
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
...,...,...,...,...,...,...,...,...,...,...,...
24778,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.67,0.00,0.33,Morally Positive
24779,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24780,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24781,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


## FASTEXT

In [3]:
data = pd.read_csv("datasets/edited/fasttext_embedded_davidson.csv")
del data["Unnamed: 0"]
data

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,0.05,-0.01,-0.02,0.02,0.04,0.00,-0.04,0.01,-0.01,-0.00,...,0.04,-0.05,0.01,0.05,0.04,0.00,0.00,0.07,0.41,0.52
1,-0.04,0.00,0.03,0.01,0.03,0.02,-0.03,-0.02,0.04,0.03,...,-0.02,0.03,-0.01,0.05,0.01,0.03,0.03,0.03,0.43,0.54
2,-0.03,-0.03,0.05,0.02,-0.01,-0.01,-0.00,-0.04,-0.02,0.03,...,-0.02,0.01,0.03,-0.02,-0.03,0.01,0.01,0.00,0.03,0.97
3,0.02,-0.02,0.02,0.02,0.02,-0.08,0.03,-0.04,-0.05,-0.02,...,0.07,-0.09,0.04,-0.04,0.06,0.01,-0.02,0.06,0.73,0.21
4,-0.04,0.00,-0.05,-0.01,0.02,-0.05,0.04,0.01,-0.03,0.02,...,0.02,0.06,0.03,-0.02,-0.03,-0.03,0.03,0.01,0.19,0.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0.03,-0.01,-0.04,0.01,-0.02,-0.06,0.00,-0.01,-0.03,0.00,...,-0.01,-0.02,0.02,0.02,-0.01,0.05,-0.02,0.01,0.12,0.87
24779,0.07,0.06,0.00,-0.06,0.02,0.02,0.03,-0.00,-0.03,0.02,...,-0.00,0.02,-0.06,0.03,0.02,-0.03,-0.02,0.01,0.10,0.89
24780,-0.05,-0.02,0.01,0.03,0.01,0.00,0.01,0.02,0.01,-0.05,...,-0.06,0.01,-0.02,0.01,0.09,-0.03,-0.03,0.04,0.39,0.57
24781,0.05,0.03,0.03,-0.03,-0.08,-0.01,0.02,-0.05,0.05,0.02,...,0.03,-0.02,0.09,-0.01,-0.02,0.01,0.10,0.01,0.11,0.88


In [4]:
data = pd.read_csv("datasets/edited/fasttext_embedded_mfrc.csv")
del data["Unnamed: 0"]
data

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,0.01,0.01,0.01,0.02,-0.01,0.00,0.01,0.02,0.01,0.01,...,-0.01,0.03,0.01,0.01,0.03,0.03,-0.01,0.11,0.32,0.57
1,-0.01,-0.01,-0.02,-0.04,0.02,0.00,-0.01,-0.03,-0.01,-0.03,...,0.03,0.04,0.02,-0.01,0.00,0.01,-0.00,0.17,0.56,0.27
2,-0.02,0.02,0.00,-0.01,-0.05,0.04,-0.02,-0.01,0.00,-0.01,...,-0.02,-0.05,0.02,-0.04,0.05,0.01,-0.03,0.07,0.45,0.49
3,-0.05,0.02,-0.01,0.03,0.01,-0.02,-0.03,-0.04,-0.03,-0.02,...,-0.03,0.05,-0.05,0.00,-0.02,-0.00,-0.01,0.02,0.35,0.63
4,0.01,-0.01,0.00,-0.00,0.01,-0.02,0.01,-0.01,-0.00,-0.03,...,0.00,0.04,0.02,0.01,0.01,-0.01,0.00,0.05,0.78,0.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,-0.01,0.02,0.01,0.01,0.02,-0.01,-0.01,0.02,-0.01,-0.02,...,-0.02,0.02,0.05,-0.01,0.02,0.02,-0.00,0.08,0.17,0.75
17882,0.02,0.01,0.02,0.01,0.02,-0.02,0.00,-0.06,0.02,-0.04,...,0.00,0.03,0.03,-0.02,0.03,0.06,0.02,0.01,0.16,0.83
17883,-0.05,-0.00,0.04,-0.04,0.02,0.02,0.02,0.01,0.01,-0.00,...,0.03,0.04,0.02,0.00,0.00,0.02,-0.00,0.02,0.25,0.73
17884,-0.00,0.02,-0.02,-0.04,-0.03,-0.02,-0.00,0.05,0.00,0.01,...,0.08,0.08,0.10,-0.01,0.05,-0.03,0.06,0.05,0.43,0.52


In [5]:
dataframe = data.join(df)
dataframe

,0,1,2,3,4,5,6,7,8,9,...,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,0.01,0.01,0.01,0.02,-0.01,0.00,0.01,0.02,0.01,0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
1,-0.01,-0.01,-0.02,-0.04,0.02,0.00,-0.01,-0.03,-0.01,-0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
2,-0.02,0.02,0.00,-0.01,-0.05,0.04,-0.02,-0.01,0.00,-0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
3,-0.05,0.02,-0.01,0.03,0.01,-0.02,-0.03,-0.04,-0.03,-0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
4,0.01,-0.01,0.00,-0.00,0.01,-0.02,0.01,-0.01,-0.00,-0.03,...,0.00,0.00,0.00,0.00,0.00,0.33,0.67,0.00,0.00,Morally Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,-0.01,0.02,0.01,0.01,0.02,-0.01,-0.01,0.02,-0.01,-0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
17882,0.02,0.01,0.02,0.01,0.02,-0.02,0.00,-0.06,0.02,-0.04,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
17883,-0.05,-0.00,0.04,-0.04,0.02,0.02,0.02,0.01,0.01,-0.00,...,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,Morally Negative
17884,-0.00,0.02,-0.02,-0.04,-0.03,-0.02,-0.00,0.05,0.00,0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [6]:
y = dataframe['target']
X = dataframe
del X['target']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.5,random_state =123)
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

100%|██████████| 29/29 [01:36<00:00,  3.34s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LinearDiscriminantAnalysis,0.98,0.98,None,0.98,0.30
DecisionTreeClassifier,0.99,0.88,None,0.99,1.98
LinearSVC,0.99,0.88,None,0.99,2.33
LogisticRegression,0.99,0.86,None,0.99,1.08
BaggingClassifier,0.99,0.85,None,0.99,11.97
Perceptron,0.98,0.80,None,0.98,0.47
CalibratedClassifierCV,0.99,0.80,None,0.99,10.34
PassiveAggressiveClassifier,0.98,0.79,None,0.98,0.46
NearestCentroid,0.78,0.79,None,0.80,0.17


In [9]:
result = pd.DataFrame(columns=["Fold","Model","Accuracy","Balanced Accuracy", "Precision-Macro",  "Precision-Weighted", "Recall-Macro", "Recall-Weighted", "F1-Score-Macro", "F1-Score-Weighted"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=42, shuffle=True)

counter = 0
foldcounter = 1
for train_index, test_index in fold.split(X, y):
  print("Processing Fold "+ str(foldcounter) + " ...")
  X_train, X_test, y_train, y_test = \
      X[ X.index.isin(train_index)], X[ X.index.isin(test_index)], y[train_index], y[test_index]
  clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
  models = clf.provide_models(X_train, X_test, y_train, y_test)
  for key, model in models.items():
    y_pred = model.predict(X_test)
    result.loc[counter]["Fold"] = foldcounter
    result.loc[counter]["Model"] = key
    result.loc[counter]["Accuracy"] = round(accuracy_score(y_test, y_pred), 3)
    result.loc[counter]["Balanced Accuracy"] = round(balanced_accuracy_score(y_test, y_pred), 3)
    result.loc[counter]["Precision-Macro"] = round(precision_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["Precision-Weighted"] = round(precision_score(y_test, y_pred, average='weighted'), 3)
    result.loc[counter]["Recall-Macro"] = round(recall_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["Recall-Weighted"] = round(recall_score(y_test, y_pred, average='weighted'), 3)
    result.loc[counter]["F1-Score-Macro"] = round(f1_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["F1-Score-Weighted"] = round(f1_score(y_test, y_pred, average='weighted'), 3)
    counter += 1
  foldcounter += 1
result = result.dropna()
result

Processing Fold 1 ...


  0%|          | 0/29 [00:00<?, ?it/s]

 90%|████████▉ | 26/29 [02:05<00:24,  8.07s/it]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004687 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26455
[LightGBM] [Info] Number of data points in the train set: 16097, number of used features: 113
[LightGBM] [Info] Start training from score -1.670070
[LightGBM] [Info] Start training from score -1.319553
[LightGBM] [Info] Start training from score -1.792443
[LightGBM] [Info] Start training from score -1.510559
[LightGBM] [Info] Start training from score -3.090608
[LightGBM] [Info] Start training from score -2.390653
[LightGBM] [Info] Start training from score -4.222556
[LightGBM] [Info] Start training from score -5.209051
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

100%|██████████| 29/29 [02:08<00:00,  4.44s/it]


...

Processing Fold 10 ...


 90%|████████▉ | 26/29 [01:57<00:21,  7.11s/it]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004354 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26454
[LightGBM] [Info] Number of data points in the train set: 16098, number of used features: 113
[LightGBM] [Info] Start training from score -1.682085
[LightGBM] [Info] Start training from score -1.304848
[LightGBM] [Info] Start training from score -1.800369
[LightGBM] [Info] Start training from score -1.514568
[LightGBM] [Info] Start training from score -3.067711
[LightGBM] [Info] Start training from score -2.392073
[LightGBM] [Info] Start training from score -4.244033
[LightGBM] [Info] Start training from score -5.232103
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

100%|██████████| 29/29 [02:00<00:00,  4.15s/it]


,Fold,Model,Accuracy,Balanced Accuracy,Precision-Macro,Precision-Weighted,Recall-Macro,Recall-Weighted,F1-Score-Macro,F1-Score-Weighted
0,1,AdaBoostClassifier,0.46,0.43,0.33,0.42,0.43,0.46,0.32,0.33
1,1,BaggingClassifier,0.97,0.93,0.94,0.97,0.93,0.97,0.94,0.97
2,1,BernoulliNB,0.87,0.77,0.74,0.88,0.77,0.87,0.75,0.88
3,1,CalibratedClassifierCV,0.97,0.81,0.97,0.97,0.81,0.97,0.84,0.97
4,1,DecisionTreeClassifier,0.97,0.94,0.91,0.97,0.94,0.97,0.92,0.97
...,...,...,...,...,...,...,...,...,...,...
245,10,RidgeClassifier,0.86,0.60,0.56,0.77,0.60,0.86,0.57,0.81
246,10,RidgeClassifierCV,0.86,0.60,0.56,0.77,0.60,0.86,0.57,0.81
247,10,SGDClassifier,0.95,0.76,0.86,0.94,0.76,0.95,0.78,0.94
248,10,SVC,0.95,0.75,0.82,0.94,0.75,0.95,0.77,0.94


In [10]:
result.to_csv("results/10-Fold-fasttext-davidson.csv")
result.to_excel("results/10-Fold-fasttext-davidson.xlsx")

In [10]:
result.to_csv("results/10-Fold-fasttext-mfrc.csv")
result.to_excel("results/10-Fold-fasttext-mfrc.xlsx")

## GLOVE

In [11]:
data = pd.read_csv("datasets/edited/glove_embedded_davidson.csv")
del data["Unnamed: 0"]
data

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,-0.11,0.31,0.35,-0.49,-0.07,0.55,-0.28,0.41,-0.01,0.26,...,-0.04,-0.28,0.10,-0.17,-0.13,0.49,-0.17,0.07,0.41,0.52
1,0.09,-0.02,0.22,-0.41,-0.36,0.09,-0.06,0.10,0.06,-0.08,...,0.25,0.12,-0.22,-0.19,-0.05,0.19,-0.33,0.03,0.43,0.54
2,0.04,0.08,0.33,-0.47,-0.28,0.48,-0.20,-0.01,0.03,-0.19,...,0.19,0.13,-0.27,-0.11,0.01,0.13,-0.03,0.00,0.03,0.97
3,-0.08,0.32,0.11,-0.39,-0.00,0.05,0.07,0.11,-0.70,-0.00,...,0.03,0.06,0.03,-0.10,-0.28,0.17,-0.08,0.06,0.73,0.21
4,0.00,0.06,0.47,-0.53,-0.34,0.24,-0.06,-0.07,0.26,-0.06,...,-0.09,0.03,-0.31,-0.37,-0.24,0.09,-0.02,0.01,0.19,0.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0.00,0.40,0.20,-0.38,0.02,0.43,0.08,0.04,-0.09,0.06,...,0.05,-0.21,-0.09,-0.31,-0.25,0.20,0.38,0.01,0.12,0.87
24779,0.08,0.12,0.50,-0.44,-0.31,0.41,-0.06,0.27,0.14,0.02,...,-0.04,-0.45,-0.27,-0.01,-0.03,0.22,0.08,0.01,0.10,0.89
24780,-0.08,0.28,0.22,-0.41,-0.41,0.17,-0.02,0.02,0.32,-0.20,...,-0.17,-0.10,-0.39,-0.12,-0.22,0.34,0.10,0.04,0.39,0.57
24781,0.04,0.13,0.80,-0.28,-0.26,0.26,0.08,0.06,0.10,-0.46,...,-0.12,-0.01,-0.20,-0.41,-0.02,0.05,0.12,0.01,0.11,0.88


In [31]:
data = pd.read_csv("datasets/edited/glove_embedded_mfrc.csv")
del data["Unnamed: 0"]
data

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,-0.21,0.27,0.40,-0.21,-0.16,0.22,-0.14,-0.18,0.20,-0.04,...,0.18,-0.19,-0.14,-0.30,-0.37,0.23,0.18,0.11,0.32,0.57
1,-0.27,0.16,0.23,-0.31,-0.09,0.32,0.00,0.05,0.06,0.01,...,0.14,-0.12,0.07,-0.13,-0.33,0.25,0.17,0.17,0.56,0.27
2,0.17,0.29,-0.04,-0.31,-0.12,0.59,0.22,-0.08,-0.31,0.18,...,0.60,0.00,-0.30,-0.54,-0.58,-0.03,-0.04,0.07,0.45,0.49
3,-0.26,0.05,0.24,-0.01,0.02,0.19,0.02,-0.25,0.22,0.20,...,0.31,-0.24,0.13,-0.23,-0.29,0.28,0.18,0.02,0.35,0.63
4,-0.11,0.24,0.06,-0.04,0.06,0.20,-0.08,-0.31,-0.12,0.13,...,0.29,-0.28,-0.07,-0.25,-0.47,0.34,0.11,0.05,0.78,0.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,0.07,0.38,0.31,-0.28,-0.32,-0.00,-0.20,0.23,0.11,-0.05,...,-0.29,-0.48,-0.01,-0.02,-0.17,0.38,0.23,0.08,0.17,0.75
17882,-0.02,0.48,0.13,-0.17,-0.53,0.34,-0.26,-0.02,0.38,-0.09,...,0.15,-0.33,0.14,0.11,-0.25,0.40,-0.30,0.01,0.16,0.83
17883,-0.27,0.10,0.27,-0.48,-0.22,0.37,0.16,-0.19,-0.36,0.09,...,0.11,-0.09,-0.14,-0.18,0.01,0.15,0.13,0.02,0.25,0.73
17884,-0.02,0.29,0.59,-0.24,-0.44,-0.03,-0.20,0.25,0.23,-0.43,...,0.30,-0.20,-0.16,0.03,-0.01,0.04,0.54,0.05,0.43,0.52


In [32]:
dataframe = data.join(df)
dataframe

,0,1,2,3,4,5,6,7,8,9,...,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,-0.21,0.27,0.40,-0.21,-0.16,0.22,-0.14,-0.18,0.20,-0.04,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
1,-0.27,0.16,0.23,-0.31,-0.09,0.32,0.00,0.05,0.06,0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
2,0.17,0.29,-0.04,-0.31,-0.12,0.59,0.22,-0.08,-0.31,0.18,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
3,-0.26,0.05,0.24,-0.01,0.02,0.19,0.02,-0.25,0.22,0.20,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
4,-0.11,0.24,0.06,-0.04,0.06,0.20,-0.08,-0.31,-0.12,0.13,...,0.00,0.00,0.00,0.00,0.00,0.33,0.67,0.00,0.00,Morally Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,0.07,0.38,0.31,-0.28,-0.32,-0.00,-0.20,0.23,0.11,-0.05,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
17882,-0.02,0.48,0.13,-0.17,-0.53,0.34,-0.26,-0.02,0.38,-0.09,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
17883,-0.27,0.10,0.27,-0.48,-0.22,0.37,0.16,-0.19,-0.36,0.09,...,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,Morally Negative
17884,-0.02,0.29,0.59,-0.24,-0.44,-0.03,-0.20,0.25,0.23,-0.43,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [33]:
y = dataframe['target']
X = dataframe
del X['target']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.5,random_state =123)
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

100%|██████████| 29/29 [01:33<00:00,  3.23s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LinearDiscriminantAnalysis,0.98,0.98,None,0.98,0.25
DecisionTreeClassifier,0.99,0.90,None,0.99,1.96
BaggingClassifier,0.99,0.85,None,0.99,11.64
LinearSVC,0.99,0.81,None,0.99,2.22
CalibratedClassifierCV,0.99,0.81,None,0.99,11.15
LogisticRegression,0.99,0.80,None,0.99,0.99
PassiveAggressiveClassifier,0.98,0.77,None,0.98,0.50
Perceptron,0.98,0.76,None,0.98,0.50
GaussianNB,0.83,0.76,None,0.86,0.19


In [34]:
result = pd.DataFrame(columns=["Fold","Model","Accuracy","Balanced Accuracy", "Precision-Macro",  "Precision-Weighted", "Recall-Macro", "Recall-Weighted", "F1-Score-Macro", "F1-Score-Weighted"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=42, shuffle=True)

counter = 0
foldcounter = 1
for train_index, test_index in fold.split(X, y):
  print("Processing Fold "+ str(foldcounter) + " ...")
  X_train, X_test, y_train, y_test = \
      X[ X.index.isin(train_index)], X[ X.index.isin(test_index)], y[train_index], y[test_index]
  clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
  models = clf.provide_models(X_train, X_test, y_train, y_test)
  for key, model in models.items():
    y_pred = model.predict(X_test)
    result.loc[counter]["Fold"] = foldcounter
    result.loc[counter]["Model"] = key
    result.loc[counter]["Accuracy"] = round(accuracy_score(y_test, y_pred), 3)
    result.loc[counter]["Balanced Accuracy"] = round(balanced_accuracy_score(y_test, y_pred), 3)
    result.loc[counter]["Precision-Macro"] = round(precision_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["Precision-Weighted"] = round(precision_score(y_test, y_pred, average='weighted'), 3)
    result.loc[counter]["Recall-Macro"] = round(recall_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["Recall-Weighted"] = round(recall_score(y_test, y_pred, average='weighted'), 3)
    result.loc[counter]["F1-Score-Macro"] = round(f1_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["F1-Score-Weighted"] = round(f1_score(y_test, y_pred, average='weighted'), 3)
    counter += 1
  foldcounter += 1
result = result.dropna()
result

Processing Fold 1 ...


 90%|████████▉ | 26/29 [02:05<00:21,  7.07s/it]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005454 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26455
[LightGBM] [Info] Number of data points in the train set: 16097, number of used features: 113
[LightGBM] [Info] Start training from score -1.670070
[LightGBM] [Info] Start training from score -1.319553
[LightGBM] [Info] Start training from score -1.792443
[LightGBM] [Info] Start training from score -1.510559
[LightGBM] [Info] Start training from score -3.090608
[LightGBM] [Info] Start training from score -2.390653
[LightGBM] [Info] Start training from score -4.222556
[LightGBM] [Info] Start training from score -5.209051
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

100%|██████████| 29/29 [02:07<00:00,  4.41s/it]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


...

 90%|████████▉ | 26/29 [01:57<00:18,  6.29s/it]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007980 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26454
[LightGBM] [Info] Number of data points in the train set: 16098, number of used features: 113
[LightGBM] [Info] Start training from score -1.682085
[LightGBM] [Info] Start training from score -1.304848
[LightGBM] [Info] Start training from score -1.800369
[LightGBM] [Info] Start training from score -1.514568
[LightGBM] [Info] Start training from score -3.067711
[LightGBM] [Info] Start training from score -2.392073
[LightGBM] [Info] Start training from score -4.244033
[LightGBM] [Info] Start training from score -5.232103
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

100%|██████████| 29/29 [01:59<00:00,  4.13s/it]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


,Fold,Model,Accuracy,Balanced Accuracy,Precision-Macro,Precision-Weighted,Recall-Macro,Recall-Weighted,F1-Score-Macro,F1-Score-Weighted
0,1,AdaBoostClassifier,0.46,0.43,0.33,0.42,0.43,0.46,0.32,0.33
1,1,BaggingClassifier,0.97,0.93,0.96,0.97,0.93,0.97,0.94,0.97
2,1,BernoulliNB,0.68,0.68,0.56,0.72,0.68,0.68,0.58,0.69
3,1,CalibratedClassifierCV,0.97,0.82,0.96,0.97,0.82,0.97,0.86,0.97
4,1,DecisionTreeClassifier,0.97,0.92,0.91,0.97,0.92,0.97,0.92,0.97
...,...,...,...,...,...,...,...,...,...,...
245,10,RidgeClassifier,0.86,0.59,0.55,0.77,0.59,0.86,0.57,0.81
246,10,RidgeClassifierCV,0.86,0.59,0.55,0.77,0.59,0.86,0.57,0.81
247,10,SGDClassifier,0.95,0.77,0.81,0.94,0.77,0.95,0.78,0.94
248,10,SVC,0.96,0.77,0.96,0.96,0.77,0.96,0.80,0.95


In [15]:
result.to_csv("results/10-Fold-glove-davidson.csv")
result.to_excel("results/10-Fold-glove-davidson.xlsx")

In [35]:
result.to_csv("results/10-Fold-glove-mfrc.csv")
result.to_excel("results/10-Fold-glove-mfrc.xlsx")

## BERT

In [16]:
data = pd.read_csv("datasets/edited/BERT_embedded_davidson.csv")
del data["Unnamed: 0"]
data

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,101,19387,9815,19454,21818,2135,2450,17612,4550,2160,...,0,0,0,0,0,0,0,0.07,0.41,0.52
1,101,19387,19875,4402,2860,2879,23755,3147,5939,3654,...,0,0,0,0,0,0,0,0.03,0.43,0.54
2,101,19387,24471,18824,11253,23544,4830,27767,19387,24829,...,0,0,0,0,0,0,0,0.00,0.03,0.97
3,101,19387,1039,1043,5143,20022,2241,2298,2066,25283,...,0,0,0,0,0,0,0,0.06,0.73,0.21
4,101,19387,21882,7556,3217,8296,2015,4485,2963,2995,...,0,0,0,0,0,0,0,0.01,0.19,0.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,101,14163,8322,2546,1999,4682,2166,19895,2075,21944,...,0,0,0,0,0,0,0,0.01,0.12,0.87
24779,101,2175,3338,3308,2540,3336,3298,2417,18278,4689,...,0,0,0,0,0,0,0,0.01,0.10,0.89
24780,101,2402,10131,10587,4521,23755,9152,13871,27225,2066,...,0,0,0,0,0,0,0,0.04,0.39,0.57
24781,101,2017,2226,2288,3748,7743,2425,2378,4682,102,...,0,0,0,0,0,0,0,0.01,0.11,0.88


In [36]:
data = pd.read_csv("datasets/edited/BERT_embedded_mfrc.csv")
del data["Unnamed: 0"]
data

,0,1,2,3,4,5,6,7,8,9,...,238,239,240,241,242,243,244,positive,neutral,negative
0,101,3327,5981,2926,6057,26632,2078,4863,2228,1042,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.11,0.32,0.57
1,101,1054,2605,3492,17133,17002,2080,2788,9969,2919,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.17,0.56,0.27
2,101,26419,2232,10115,3393,7279,2092,9346,14870,5443,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.07,0.45,0.49
3,101,5866,3663,6039,2239,6771,3747,14303,7731,2283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.02,0.35,0.63
4,101,3393,7279,4435,9530,8043,20203,6491,4556,2157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.05,0.78,0.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,101,3105,2941,7929,2147,7170,3835,2111,11519,3477,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.08,0.17,0.75
17882,101,3280,4295,2689,5294,2135,10509,12765,2111,4452,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.01,0.16,0.83
17883,101,3761,12268,2202,10048,7226,2368,2655,2317,22006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.02,0.25,0.73
17884,101,3606,2755,3524,7226,2368,102,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.05,0.43,0.52


In [37]:
dataframe = data.join(df)
dataframe

,0,1,2,3,4,5,6,7,8,9,...,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,101,3327,5981,2926,6057,26632,2078,4863,2228,1042,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
1,101,1054,2605,3492,17133,17002,2080,2788,9969,2919,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
2,101,26419,2232,10115,3393,7279,2092,9346,14870,5443,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
3,101,5866,3663,6039,2239,6771,3747,14303,7731,2283,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
4,101,3393,7279,4435,9530,8043,20203,6491,4556,2157,...,0.00,0.00,0.00,0.00,0.00,0.33,0.67,0.00,0.00,Morally Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,101,3105,2941,7929,2147,7170,3835,2111,11519,3477,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
17882,101,3280,4295,2689,5294,2135,10509,12765,2111,4452,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
17883,101,3761,12268,2202,10048,7226,2368,2655,2317,22006,...,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,Morally Negative
17884,101,3606,2755,3524,7226,2368,102,0,0,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [38]:
y = dataframe['target']
X = dataframe
del X['target']

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.5,random_state =123)
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

100%|██████████| 29/29 [01:35<00:00,  3.30s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LinearDiscriminantAnalysis,0.98,0.99,None,0.99,0.24
BaggingClassifier,0.99,0.92,None,0.99,1.66
DecisionTreeClassifier,0.99,0.90,None,0.99,0.36
LinearSVC,0.99,0.89,None,0.99,3.37
LogisticRegression,0.99,0.88,None,0.99,0.87
CalibratedClassifierCV,0.99,0.85,None,0.99,46.08
PassiveAggressiveClassifier,0.98,0.85,None,0.98,0.45
Perceptron,0.98,0.84,None,0.98,0.41
NearestCentroid,0.86,0.83,None,0.87,0.19


In [39]:
result = pd.DataFrame(columns=["Fold","Model","Accuracy","Balanced Accuracy", "Precision-Macro",  "Precision-Weighted", "Recall-Macro", "Recall-Weighted", "F1-Score-Macro", "F1-Score-Weighted"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=42, shuffle=True)

counter = 0
foldcounter = 1
for train_index, test_index in fold.split(X, y):
  print("Processing Fold "+ str(foldcounter) + " ...")
  X_train, X_test, y_train, y_test = \
      X[ X.index.isin(train_index)], X[ X.index.isin(test_index)], y[train_index], y[test_index]
  clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
  models = clf.provide_models(X_train, X_test, y_train, y_test)
  for key, model in models.items():
    y_pred = model.predict(X_test)
    result.loc[counter]["Fold"] = foldcounter
    result.loc[counter]["Model"] = key
    result.loc[counter]["Accuracy"] = round(accuracy_score(y_test, y_pred), 3)
    result.loc[counter]["Balanced Accuracy"] = round(balanced_accuracy_score(y_test, y_pred), 3)
    result.loc[counter]["Precision-Macro"] = round(precision_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["Precision-Weighted"] = round(precision_score(y_test, y_pred, average='weighted'), 3)
    result.loc[counter]["Recall-Macro"] = round(recall_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["Recall-Weighted"] = round(recall_score(y_test, y_pred, average='weighted'), 3)
    result.loc[counter]["F1-Score-Macro"] = round(f1_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["F1-Score-Weighted"] = round(f1_score(y_test, y_pred, average='weighted'), 3)
    counter += 1
  foldcounter += 1
result = result.dropna()
result

Processing Fold 1 ...


  0%|          | 0/29 [00:00<?, ?it/s]

 97%|█████████▋| 28/29 [09:05<00:03,  3.44s/it]  

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003730 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 15174
[LightGBM] [Info] Number of data points in the train set: 16097, number of used features: 112
[LightGBM] [Info] Start training from score -1.670070
[LightGBM] [Info] Start training from score -1.319553
[LightGBM] [Info] Start training from score -1.792443
[LightGBM] [Info] Start training from score -1.510559
[LightGBM] [Info] Start training from score -3.090608
[LightGBM] [Info] Start training from score -2.390653
[LightGBM] [Info] Start training from score -4.222556
[LightGBM] [Info] Start training from score -5.209051
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

100%|██████████| 29/29 [09:08<00:00, 18.90s/it]


...

 97%|█████████▋| 28/29 [08:44<00:03,  3.44s/it]  

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014383 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15202
[LightGBM] [Info] Number of data points in the train set: 16098, number of used features: 112
[LightGBM] [Info] Start training from score -1.682085
[LightGBM] [Info] Start training from score -1.304848
[LightGBM] [Info] Start training from score -1.800369
[LightGBM] [Info] Start training from score -1.514568
[LightGBM] [Info] Start training from score -3.067711
[LightGBM] [Info] Start training from score -2.392073
[LightGBM] [Info] Start training from score -4.244033
[LightGBM] [Info] Start training from score -5.232103
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

100%|██████████| 29/29 [08:47<00:00, 18.17s/it]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

,Fold,Model,Accuracy,Balanced Accuracy,Precision-Macro,Precision-Weighted,Recall-Macro,Recall-Weighted,F1-Score-Macro,F1-Score-Weighted
0,1,AdaBoostClassifier,0.46,0.43,0.33,0.42,0.43,0.46,0.32,0.33
1,1,BaggingClassifier,0.98,0.95,0.97,0.98,0.95,0.98,0.96,0.98
2,1,BernoulliNB,0.55,0.49,0.44,0.64,0.49,0.55,0.42,0.57
3,1,CalibratedClassifierCV,0.96,0.72,0.73,0.94,0.72,0.96,0.72,0.95
4,1,DecisionTreeClassifier,0.97,0.92,0.93,0.97,0.92,0.97,0.92,0.97
...,...,...,...,...,...,...,...,...,...,...
245,10,RidgeClassifier,0.86,0.60,0.68,0.86,0.60,0.86,0.58,0.82
246,10,RidgeClassifierCV,0.86,0.60,0.68,0.86,0.60,0.86,0.57,0.82
247,10,SGDClassifier,0.95,0.84,0.91,0.95,0.84,0.95,0.87,0.95
248,10,SVC,0.94,0.73,0.80,0.94,0.73,0.94,0.74,0.94


In [20]:
result.to_csv("results/10-Fold-BERT-davidson.csv")
result.to_excel("results/10-Fold-BERT-davidson.xlsx")

In [40]:
result.to_csv("results/10-Fold-BERT-mfrc.csv")
result.to_excel("results/10-Fold-BERT-mfrc.xlsx")

## WORD2VEC

In [21]:
data = pd.read_csv("datasets/edited/word2vec_embedded_davidson.csv")
del data["Unnamed: 0"]
data

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,0.04,0.13,0.04,-0.05,-0.02,0.03,-0.06,-0.05,0.10,0.06,...,-0.10,-0.06,0.06,-0.09,0.02,0.04,-0.05,0.07,0.41,0.52
1,-0.01,0.15,0.05,0.05,-0.06,-0.02,-0.05,-0.10,0.02,0.09,...,-0.19,-0.01,0.07,0.06,-0.00,0.16,-0.07,0.03,0.43,0.54
2,-0.02,0.04,0.10,0.16,-0.11,0.05,-0.04,-0.20,0.10,0.07,...,-0.16,-0.10,0.08,-0.00,0.04,0.03,-0.08,0.00,0.03,0.97
3,-0.16,0.12,0.09,0.16,-0.05,0.02,-0.01,-0.17,-0.08,0.03,...,-0.02,-0.02,0.12,-0.02,-0.02,0.14,-0.05,0.06,0.73,0.21
4,0.05,-0.03,0.10,0.11,-0.13,0.11,-0.01,-0.07,0.13,0.09,...,-0.05,-0.12,0.04,0.09,0.12,0.11,-0.16,0.01,0.19,0.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0.08,-0.04,0.11,0.15,0.03,-0.05,-0.01,-0.04,0.03,0.17,...,-0.12,-0.13,0.03,-0.01,-0.07,0.07,-0.11,0.01,0.12,0.87
24779,-0.01,0.03,-0.02,0.09,-0.09,0.04,0.07,-0.04,0.04,0.13,...,-0.13,-0.09,0.01,0.14,0.03,-0.05,0.07,0.01,0.10,0.89
24780,-0.04,0.04,0.02,0.18,-0.06,0.04,0.01,-0.04,0.03,0.12,...,-0.16,-0.00,-0.05,0.15,0.06,0.05,-0.17,0.04,0.39,0.57
24781,0.01,0.01,0.08,0.06,0.02,-0.09,-0.02,0.00,0.00,0.21,...,-0.06,-0.15,0.00,0.10,0.04,0.13,-0.13,0.01,0.11,0.88


In [41]:
data = pd.read_csv("datasets/edited/word2vec_embedded_mfrc.csv")
del data["Unnamed: 0"]
data

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,0.07,0.01,-0.02,0.12,-0.14,0.03,0.08,-0.01,0.10,0.03,...,-0.00,0.06,-0.08,0.11,0.06,0.04,-0.02,0.11,0.32,0.57
1,0.03,0.05,-0.03,0.14,-0.08,0.04,0.00,-0.04,0.05,0.02,...,-0.08,0.00,-0.04,0.03,-0.01,0.07,-0.04,0.17,0.56,0.27
2,0.04,0.03,0.06,0.09,-0.02,0.12,-0.04,-0.06,0.16,-0.01,...,-0.07,-0.06,-0.03,0.08,-0.03,-0.02,-0.10,0.07,0.45,0.49
3,0.13,0.03,-0.09,0.05,-0.12,0.01,0.02,-0.06,0.04,-0.01,...,0.07,-0.02,-0.06,0.09,0.05,0.01,0.00,0.02,0.35,0.63
4,0.07,0.00,0.02,0.16,-0.07,-0.01,0.04,-0.05,0.12,-0.00,...,-0.06,-0.04,-0.06,0.04,-0.06,0.00,-0.07,0.05,0.78,0.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,0.07,0.06,-0.02,0.09,-0.03,0.03,0.11,-0.04,0.08,0.06,...,-0.09,-0.02,-0.12,0.02,0.09,0.04,-0.07,0.08,0.17,0.75
17882,0.12,0.06,0.02,0.17,-0.20,0.07,0.05,-0.08,0.13,0.06,...,-0.07,0.05,-0.07,0.14,-0.08,0.11,-0.03,0.01,0.16,0.83
17883,0.07,0.00,0.11,0.03,0.01,-0.05,-0.02,-0.05,0.09,0.03,...,-0.06,-0.01,-0.02,0.06,0.08,0.10,-0.00,0.02,0.25,0.73
17884,0.12,0.07,0.09,0.08,-0.04,0.06,0.10,0.02,0.21,0.11,...,0.02,-0.14,0.01,0.04,0.07,0.15,-0.15,0.05,0.43,0.52


In [42]:
dataframe = data.join(df)
dataframe

,0,1,2,3,4,5,6,7,8,9,...,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,0.07,0.01,-0.02,0.12,-0.14,0.03,0.08,-0.01,0.10,0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
1,0.03,0.05,-0.03,0.14,-0.08,0.04,0.00,-0.04,0.05,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
2,0.04,0.03,0.06,0.09,-0.02,0.12,-0.04,-0.06,0.16,-0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
3,0.13,0.03,-0.09,0.05,-0.12,0.01,0.02,-0.06,0.04,-0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
4,0.07,0.00,0.02,0.16,-0.07,-0.01,0.04,-0.05,0.12,-0.00,...,0.00,0.00,0.00,0.00,0.00,0.33,0.67,0.00,0.00,Morally Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,0.07,0.06,-0.02,0.09,-0.03,0.03,0.11,-0.04,0.08,0.06,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
17882,0.12,0.06,0.02,0.17,-0.20,0.07,0.05,-0.08,0.13,0.06,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
17883,0.07,0.00,0.11,0.03,0.01,-0.05,-0.02,-0.05,0.09,0.03,...,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,Morally Negative
17884,0.12,0.07,0.09,0.08,-0.04,0.06,0.10,0.02,0.21,0.11,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [43]:
y = dataframe['target']
X = dataframe
del X['target']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.5,random_state =123)
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

  0%|          | 0/29 [00:00<?, ?it/s]

100%|██████████| 29/29 [01:34<00:00,  3.25s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LinearDiscriminantAnalysis,0.98,0.98,None,0.98,0.26
LGBMClassifier,0.99,0.87,None,0.99,3.86
LinearSVC,0.99,0.86,None,0.99,2.38
DecisionTreeClassifier,0.99,0.86,None,0.99,1.91
BaggingClassifier,0.99,0.84,None,0.99,11.57
LogisticRegression,0.99,0.83,None,0.99,1.18
CalibratedClassifierCV,0.99,0.82,None,0.99,12.50
Perceptron,0.98,0.80,None,0.98,0.47
PassiveAggressiveClassifier,0.98,0.79,None,0.98,0.50


In [44]:
result = pd.DataFrame(columns=["Fold","Model","Accuracy","Balanced Accuracy", "Precision-Macro",  "Precision-Weighted", "Recall-Macro", "Recall-Weighted", "F1-Score-Macro", "F1-Score-Weighted"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=42, shuffle=True)

counter = 0
foldcounter = 1
for train_index, test_index in fold.split(X, y):
  print("Processing Fold "+ str(foldcounter) + " ...")
  X_train, X_test, y_train, y_test = \
      X[ X.index.isin(train_index)], X[ X.index.isin(test_index)], y[train_index], y[test_index]
  clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
  models = clf.provide_models(X_train, X_test, y_train, y_test)
  for key, model in models.items():
    y_pred = model.predict(X_test)
    result.loc[counter]["Fold"] = foldcounter
    result.loc[counter]["Model"] = key
    result.loc[counter]["Accuracy"] = round(accuracy_score(y_test, y_pred), 3)
    result.loc[counter]["Balanced Accuracy"] = round(balanced_accuracy_score(y_test, y_pred), 3)
    result.loc[counter]["Precision-Macro"] = round(precision_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["Precision-Weighted"] = round(precision_score(y_test, y_pred, average='weighted'), 3)
    result.loc[counter]["Recall-Macro"] = round(recall_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["Recall-Weighted"] = round(recall_score(y_test, y_pred, average='weighted'), 3)
    result.loc[counter]["F1-Score-Macro"] = round(f1_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["F1-Score-Weighted"] = round(f1_score(y_test, y_pred, average='weighted'), 3)
    counter += 1
  foldcounter += 1
result = result.dropna()
result

Processing Fold 1 ...


  0%|          | 0/29 [00:00<?, ?it/s]

 90%|████████▉ | 26/29 [01:54<00:19,  6.39s/it]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004506 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26455
[LightGBM] [Info] Number of data points in the train set: 16097, number of used features: 113
[LightGBM] [Info] Start training from score -1.670070
[LightGBM] [Info] Start training from score -1.319553
[LightGBM] [Info] Start training from score -1.792443
[LightGBM] [Info] Start training from score -1.510559
[LightGBM] [Info] Start training from score -3.090608
[LightGBM] [Info] Start training from score -2.390653
[LightGBM] [Info] Start training from score -4.222556
[LightGBM] [Info] Start training from score -5.209051
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

100%|██████████| 29/29 [01:56<00:00,  4.03s/it]

...

 90%|████████▉ | 26/29 [01:55<00:19,  6.53s/it]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004932 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26454
[LightGBM] [Info] Number of data points in the train set: 16098, number of used features: 113
[LightGBM] [Info] Start training from score -1.682085
[LightGBM] [Info] Start training from score -1.304848
[LightGBM] [Info] Start training from score -1.800369
[LightGBM] [Info] Start training from score -1.514568
[LightGBM] [Info] Start training from score -3.067711
[LightGBM] [Info] Start training from score -2.392073
[LightGBM] [Info] Start training from score -4.244033
[LightGBM] [Info] Start training from score -5.232103
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

100%|██████████| 29/29 [01:57<00:00,  4.06s/it]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


,Fold,Model,Accuracy,Balanced Accuracy,Precision-Macro,Precision-Weighted,Recall-Macro,Recall-Weighted,F1-Score-Macro,F1-Score-Weighted
0,1,AdaBoostClassifier,0.46,0.43,0.33,0.42,0.43,0.46,0.32,0.33
1,1,BaggingClassifier,0.97,0.92,0.96,0.97,0.92,0.97,0.94,0.97
2,1,BernoulliNB,0.75,0.72,0.61,0.78,0.72,0.75,0.64,0.76
3,1,CalibratedClassifierCV,0.97,0.81,0.96,0.97,0.81,0.97,0.84,0.97
4,1,DecisionTreeClassifier,0.97,0.94,0.93,0.97,0.94,0.97,0.93,0.97
...,...,...,...,...,...,...,...,...,...,...
245,10,RidgeClassifier,0.86,0.59,0.68,0.86,0.59,0.86,0.57,0.81
246,10,RidgeClassifierCV,0.86,0.59,0.68,0.86,0.59,0.86,0.57,0.81
247,10,SGDClassifier,0.95,0.77,0.81,0.94,0.77,0.95,0.78,0.94
248,10,SVC,0.95,0.75,0.94,0.95,0.75,0.95,0.78,0.94


In [26]:
result.to_csv("results/10-Fold-word2vec-davidson.csv")
result.to_excel("results/10-Fold-word2vec-davidson.xlsx")

In [45]:
result.to_csv("results/10-Fold-word2vec-mfrc.csv")
result.to_excel("results/10-Fold-word2vec-mfrc.xlsx")

## TFDIF


In [26]:
data = pd.read_csv("datasets/edited/tfdif_embedded_davidson.csv")
del data["Unnamed: 0"]
data

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,0.45,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.58,0.00,0.00,0.00,0.00,0.00,0.00,0.07,0.41,0.52
1,0.00,0.00,0.35,0.00,0.00,0.00,0.00,0.00,0.00,0.41,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.43,0.54
2,0.00,0.00,0.00,0.00,0.00,0.00,0.21,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.97
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.06,0.73,0.21
4,0.00,0.00,0.00,0.00,0.00,0.00,0.22,0.00,0.00,0.00,...,0.00,0.00,0.59,0.00,0.00,0.00,0.00,0.01,0.19,0.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.12,0.87
24779,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.10,0.89
24780,0.00,0.00,0.00,0.32,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.04,0.39,0.57
24781,0.00,0.00,0.00,0.00,0.00,0.00,0.36,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.11,0.88


In [46]:
data = pd.read_csv("datasets/edited/tfdif_embedded_mfrc.csv")
del data["Unnamed: 0"]
data

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.11,0.32,0.57
1,0.00,0.00,0.29,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.17,0.56,0.27
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.07,0.45,0.49
3,0.00,0.00,0.00,0.00,0.00,0.51,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.35,0.63
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.05,0.78,0.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,0.26,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.47,0.00,0.00,0.00,0.00,0.00,0.08,0.17,0.75
17882,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.85,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.16,0.83
17883,0.00,0.00,0.00,0.00,0.57,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.25,0.73
17884,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.05,0.43,0.52


In [47]:
dataframe = data.join(df)
dataframe

,0,1,2,3,4,5,6,7,8,9,...,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
1,0.00,0.00,0.29,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
3,0.00,0.00,0.00,0.00,0.00,0.51,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.33,0.67,0.00,0.00,Morally Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,0.26,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
17882,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.85,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
17883,0.00,0.00,0.00,0.00,0.57,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,Morally Negative
17884,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [48]:
y = dataframe['target']
X = dataframe
del X['target']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.5,random_state =123)
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

100%|██████████| 29/29 [01:05<00:00,  2.25s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LinearDiscriminantAnalysis,0.98,0.98,None,0.98,0.28
DecisionTreeClassifier,0.99,0.90,None,0.99,0.27
ExtraTreesClassifier,0.99,0.89,None,0.99,1.65
BaggingClassifier,0.99,0.87,None,0.99,0.95
RandomForestClassifier,0.99,0.86,None,0.99,2.26
PassiveAggressiveClassifier,0.98,0.83,None,0.98,0.46
LogisticRegression,0.99,0.82,None,0.99,0.70
LinearSVC,0.99,0.81,None,0.99,2.32
CalibratedClassifierCV,0.99,0.79,None,0.99,13.10


In [49]:
result = pd.DataFrame(columns=["Fold","Model","Accuracy","Balanced Accuracy", "Precision-Macro",  "Precision-Weighted", "Recall-Macro", "Recall-Weighted", "F1-Score-Macro", "F1-Score-Weighted"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=42, shuffle=True)

counter = 0
foldcounter = 1
for train_index, test_index in fold.split(X, y):
  print("Processing Fold "+ str(foldcounter) + " ...")
  X_train, X_test, y_train, y_test = \
      X[ X.index.isin(train_index)], X[ X.index.isin(test_index)], y[train_index], y[test_index]
  clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
  models = clf.provide_models(X_train, X_test, y_train, y_test)
  for key, model in models.items():
    y_pred = model.predict(X_test)
    result.loc[counter]["Fold"] = foldcounter
    result.loc[counter]["Model"] = key
    result.loc[counter]["Accuracy"] = round(accuracy_score(y_test, y_pred), 3)
    result.loc[counter]["Balanced Accuracy"] = round(balanced_accuracy_score(y_test, y_pred), 3)
    result.loc[counter]["Precision-Macro"] = round(precision_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["Precision-Weighted"] = round(precision_score(y_test, y_pred, average='weighted'), 3)
    result.loc[counter]["Recall-Macro"] = round(recall_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["Recall-Weighted"] = round(recall_score(y_test, y_pred, average='weighted'), 3)
    result.loc[counter]["F1-Score-Macro"] = round(f1_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["F1-Score-Weighted"] = round(f1_score(y_test, y_pred, average='weighted'), 3)
    counter += 1
  foldcounter += 1
result = result.dropna()
result

Processing Fold 1 ...


  0%|          | 0/29 [00:00<?, ?it/s]

 90%|████████▉ | 26/29 [01:11<00:15,  5.12s/it]

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004604 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 18004
[LightGBM] [Info] Number of data points in the train set: 16097, number of used features: 113
[LightGBM] [Info] Start training from score -1.670070
[LightGBM] [Info] Start training from score -1.319553
[LightGBM] [Info] Start training from score -1.792443
[LightGBM] [Info] Start training from score -1.510559
[LightGBM] [Info] Start training from score -3.090608
[LightGBM] [Info] Start training from score -2.390653
[LightGBM] [Info] Start training from score -4.222556
[LightGBM] [Info] Start training from score -5.209051
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

100%|██████████| 29/29 [01:12<00:00,  2.51s/it]

...

Processing Fold 10 ...


 90%|████████▉ | 26/29 [01:11<00:15,  5.07s/it]

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005365 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 17980
[LightGBM] [Info] Number of data points in the train set: 16098, number of used features: 113
[LightGBM] [Info] Start training from score -1.682085
[LightGBM] [Info] Start training from score -1.304848
[LightGBM] [Info] Start training from score -1.800369
[LightGBM] [Info] Start training from score -1.514568
[LightGBM] [Info] Start training from score -3.067711
[LightGBM] [Info] Start training from score -2.392073
[LightGBM] [Info] Start training from score -4.244033
[LightGBM] [Info] Start training from score -5.232103
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

100%|██████████| 29/29 [01:13<00:00,  2.52s/it]


,Fold,Model,Accuracy,Balanced Accuracy,Precision-Macro,Precision-Weighted,Recall-Macro,Recall-Weighted,F1-Score-Macro,F1-Score-Weighted
0,1,AdaBoostClassifier,0.46,0.43,0.33,0.42,0.43,0.46,0.32,0.33
1,1,BaggingClassifier,0.98,0.95,0.97,0.98,0.95,0.98,0.96,0.98
2,1,BernoulliNB,0.86,0.76,0.76,0.87,0.76,0.86,0.76,0.86
3,1,CalibratedClassifierCV,0.97,0.84,0.94,0.97,0.84,0.97,0.88,0.97
4,1,DecisionTreeClassifier,0.98,0.94,0.95,0.98,0.94,0.98,0.95,0.98
...,...,...,...,...,...,...,...,...,...,...
245,10,RidgeClassifier,0.86,0.60,0.63,0.83,0.60,0.86,0.58,0.82
246,10,RidgeClassifierCV,0.86,0.60,0.63,0.83,0.60,0.86,0.58,0.82
247,10,SGDClassifier,0.94,0.75,0.80,0.94,0.75,0.94,0.77,0.94
248,10,SVC,0.95,0.73,0.96,0.95,0.73,0.95,0.74,0.94


In [31]:
result.to_csv("results/10-Fold-tfdif-davidson.csv")
result.to_excel("results/10-Fold-tfdif-davidson.xlsx")

In [50]:
result.to_csv("results/10-Fold-tfdif-mfrc.csv")
result.to_excel("results/10-Fold-tfdif-mfrc.xlsx")

## DAVIDSON + MFRC 


In [3]:
df = pd.read_csv("datasets/edited/davidson_with_classes.csv")
del df['positive']
del df['negative']
del df['neutral']
df

,care.virtue,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,1.00,Partially Negative
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
...,...,...,...,...,...,...,...,...,...,...,...
24778,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.67,0.00,0.33,Morally Positive
24779,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24780,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24781,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [4]:
df2 = pd.read_csv("datasets/edited/mfrc_with_classes.csv")
del df2['positive']
del df2['negative']
del df2['neutral']
df2

,care.virtue,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
4,0.00,0.00,0.00,0.00,0.00,0.00,0.33,0.67,0.00,0.00,Morally Positive
...,...,...,...,...,...,...,...,...,...,...,...
17881,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
17882,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
17883,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,Morally Negative
17884,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


## FASTEXT

In [5]:
data1 = pd.read_csv("datasets/edited/fasttext_embedded_davidson.csv")
del data1["Unnamed: 0"]
data1

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,0.05,-0.01,-0.02,0.02,0.04,0.00,-0.04,0.01,-0.01,-0.00,...,0.04,-0.05,0.01,0.05,0.04,0.00,0.00,0.07,0.41,0.52
1,-0.04,0.00,0.03,0.01,0.03,0.02,-0.03,-0.02,0.04,0.03,...,-0.02,0.03,-0.01,0.05,0.01,0.03,0.03,0.03,0.43,0.54
2,-0.03,-0.03,0.05,0.02,-0.01,-0.01,-0.00,-0.04,-0.02,0.03,...,-0.02,0.01,0.03,-0.02,-0.03,0.01,0.01,0.00,0.03,0.97
3,0.02,-0.02,0.02,0.02,0.02,-0.08,0.03,-0.04,-0.05,-0.02,...,0.07,-0.09,0.04,-0.04,0.06,0.01,-0.02,0.06,0.73,0.21
4,-0.04,0.00,-0.05,-0.01,0.02,-0.05,0.04,0.01,-0.03,0.02,...,0.02,0.06,0.03,-0.02,-0.03,-0.03,0.03,0.01,0.19,0.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0.03,-0.01,-0.04,0.01,-0.02,-0.06,0.00,-0.01,-0.03,0.00,...,-0.01,-0.02,0.02,0.02,-0.01,0.05,-0.02,0.01,0.12,0.87
24779,0.07,0.06,0.00,-0.06,0.02,0.02,0.03,-0.00,-0.03,0.02,...,-0.00,0.02,-0.06,0.03,0.02,-0.03,-0.02,0.01,0.10,0.89
24780,-0.05,-0.02,0.01,0.03,0.01,0.00,0.01,0.02,0.01,-0.05,...,-0.06,0.01,-0.02,0.01,0.09,-0.03,-0.03,0.04,0.39,0.57
24781,0.05,0.03,0.03,-0.03,-0.08,-0.01,0.02,-0.05,0.05,0.02,...,0.03,-0.02,0.09,-0.01,-0.02,0.01,0.10,0.01,0.11,0.88


In [6]:
data2 = pd.read_csv("datasets/edited/fasttext_embedded_mfrc.csv")
del data2["Unnamed: 0"]
data2

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,0.01,0.01,0.01,0.02,-0.01,0.00,0.01,0.02,0.01,0.01,...,-0.01,0.03,0.01,0.01,0.03,0.03,-0.01,0.11,0.32,0.57
1,-0.01,-0.01,-0.02,-0.04,0.02,0.00,-0.01,-0.03,-0.01,-0.03,...,0.03,0.04,0.02,-0.01,0.00,0.01,-0.00,0.17,0.56,0.27
2,-0.02,0.02,0.00,-0.01,-0.05,0.04,-0.02,-0.01,0.00,-0.01,...,-0.02,-0.05,0.02,-0.04,0.05,0.01,-0.03,0.07,0.45,0.49
3,-0.05,0.02,-0.01,0.03,0.01,-0.02,-0.03,-0.04,-0.03,-0.02,...,-0.03,0.05,-0.05,0.00,-0.02,-0.00,-0.01,0.02,0.35,0.63
4,0.01,-0.01,0.00,-0.00,0.01,-0.02,0.01,-0.01,-0.00,-0.03,...,0.00,0.04,0.02,0.01,0.01,-0.01,0.00,0.05,0.78,0.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,-0.01,0.02,0.01,0.01,0.02,-0.01,-0.01,0.02,-0.01,-0.02,...,-0.02,0.02,0.05,-0.01,0.02,0.02,-0.00,0.08,0.17,0.75
17882,0.02,0.01,0.02,0.01,0.02,-0.02,0.00,-0.06,0.02,-0.04,...,0.00,0.03,0.03,-0.02,0.03,0.06,0.02,0.01,0.16,0.83
17883,-0.05,-0.00,0.04,-0.04,0.02,0.02,0.02,0.01,0.01,-0.00,...,0.03,0.04,0.02,0.00,0.00,0.02,-0.00,0.02,0.25,0.73
17884,-0.00,0.02,-0.02,-0.04,-0.03,-0.02,-0.00,0.05,0.00,0.01,...,0.08,0.08,0.10,-0.01,0.05,-0.03,0.06,0.05,0.43,0.52


In [7]:
data = data1.join(df)
data

,0,1,2,3,4,5,6,7,8,9,...,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,0.05,-0.01,-0.02,0.02,0.04,0.00,-0.04,0.01,-0.01,-0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,1.00,Partially Negative
1,-0.04,0.00,0.03,0.01,0.03,0.02,-0.03,-0.02,0.04,0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
2,-0.03,-0.03,0.05,0.02,-0.01,-0.01,-0.00,-0.04,-0.02,0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
3,0.02,-0.02,0.02,0.02,0.02,-0.08,0.03,-0.04,-0.05,-0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
4,-0.04,0.00,-0.05,-0.01,0.02,-0.05,0.04,0.01,-0.03,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0.03,-0.01,-0.04,0.01,-0.02,-0.06,0.00,-0.01,-0.03,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.67,0.00,0.33,Morally Positive
24779,0.07,0.06,0.00,-0.06,0.02,0.02,0.03,-0.00,-0.03,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24780,-0.05,-0.02,0.01,0.03,0.01,0.00,0.01,0.02,0.01,-0.05,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24781,0.05,0.03,0.03,-0.03,-0.08,-0.01,0.02,-0.05,0.05,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [8]:
dataframe2 = data2.join(df2)
dataframe2 

,0,1,2,3,4,5,6,7,8,9,...,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,0.01,0.01,0.01,0.02,-0.01,0.00,0.01,0.02,0.01,0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
1,-0.01,-0.01,-0.02,-0.04,0.02,0.00,-0.01,-0.03,-0.01,-0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
2,-0.02,0.02,0.00,-0.01,-0.05,0.04,-0.02,-0.01,0.00,-0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
3,-0.05,0.02,-0.01,0.03,0.01,-0.02,-0.03,-0.04,-0.03,-0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
4,0.01,-0.01,0.00,-0.00,0.01,-0.02,0.01,-0.01,-0.00,-0.03,...,0.00,0.00,0.00,0.00,0.00,0.33,0.67,0.00,0.00,Morally Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,-0.01,0.02,0.01,0.01,0.02,-0.01,-0.01,0.02,-0.01,-0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
17882,0.02,0.01,0.02,0.01,0.02,-0.02,0.00,-0.06,0.02,-0.04,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
17883,-0.05,-0.00,0.04,-0.04,0.02,0.02,0.02,0.01,0.01,-0.00,...,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,Morally Negative
17884,-0.00,0.02,-0.02,-0.04,-0.03,-0.02,-0.00,0.05,0.00,0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [9]:
dataframe = pd.concat([data, dataframe2])
dataframe = dataframe.reset_index()
dataframe

,index,0,1,2,3,4,5,6,7,8,...,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,0,0.05,-0.01,-0.02,0.02,0.04,0.00,-0.04,0.01,-0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,1.00,Partially Negative
1,1,-0.04,0.00,0.03,0.01,0.03,0.02,-0.03,-0.02,0.04,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
2,2,-0.03,-0.03,0.05,0.02,-0.01,-0.01,-0.00,-0.04,-0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
3,3,0.02,-0.02,0.02,0.02,0.02,-0.08,0.03,-0.04,-0.05,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
4,4,-0.04,0.00,-0.05,-0.01,0.02,-0.05,0.04,0.01,-0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42664,17881,-0.01,0.02,0.01,0.01,0.02,-0.01,-0.01,0.02,-0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
42665,17882,0.02,0.01,0.02,0.01,0.02,-0.02,0.00,-0.06,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
42666,17883,-0.05,-0.00,0.04,-0.04,0.02,0.02,0.02,0.01,0.01,...,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,Morally Negative
42667,17884,-0.00,0.02,-0.02,-0.04,-0.03,-0.02,-0.00,0.05,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [10]:
y = dataframe['target']
X = dataframe
del X['target']

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.5,random_state =123)
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

  0%|          | 0/29 [00:00<?, ?it/s]

 90%|████████▉ | 26/29 [03:43<00:49, 16.58s/it]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007486 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26675
[LightGBM] [Info] Number of data points in the train set: 21334, number of used features: 114
[LightGBM] [Info] Start training from score -1.563809
[LightGBM] [Info] Start training from score -1.810400
[LightGBM] [Info] Start training from score -1.607986
[LightGBM] [Info] Start training from score -1.102746
[LightGBM] [Info] Start training from score -3.408442
[LightGBM] [Info] Start training from score -2.970461
[LightGBM] [Info] Start training from score -4.856070
[LightGBM] [Info] Start training from score -5.925006
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

100%|██████████| 29/29 [03:46<00:00,  7.81s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LinearDiscriminantAnalysis,0.97,0.95,None,0.97,0.67
LGBMClassifier,0.99,0.94,None,0.99,3.08
LogisticRegression,0.99,0.90,None,0.99,1.95
GaussianNB,0.91,0.90,None,0.91,0.34
BaggingClassifier,0.98,0.89,None,0.98,21.96
DecisionTreeClassifier,0.98,0.88,None,0.98,3.75
LinearSVC,0.98,0.86,None,0.98,17.29
CalibratedClassifierCV,0.98,0.84,None,0.98,12.75
Perceptron,0.97,0.83,None,0.97,0.99


In [11]:
result = pd.DataFrame(columns=["Fold","Model","Accuracy","Balanced Accuracy", "Precision-Macro",  "Precision-Weighted", "Recall-Macro", "Recall-Weighted", "F1-Score-Macro", "F1-Score-Weighted"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=42, shuffle=True)

counter = 0
foldcounter = 1
for train_index, test_index in fold.split(X, y):
  print("Processing Fold "+ str(foldcounter) + " ...")
  X_train, X_test, y_train, y_test = \
      X[ X.index.isin(train_index)], X[ X.index.isin(test_index)], y[train_index], y[test_index]
  clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
  models = clf.provide_models(X_train, X_test, y_train, y_test)
  for key, model in models.items():
    y_pred = model.predict(X_test)
    result.loc[counter]["Fold"] = foldcounter
    result.loc[counter]["Model"] = key
    result.loc[counter]["Accuracy"] = round(accuracy_score(y_test, y_pred), 3)
    result.loc[counter]["Balanced Accuracy"] = round(balanced_accuracy_score(y_test, y_pred), 3)
    result.loc[counter]["Precision-Macro"] = round(precision_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["Precision-Weighted"] = round(precision_score(y_test, y_pred, average='weighted'), 3)
    result.loc[counter]["Recall-Macro"] = round(recall_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["Recall-Weighted"] = round(recall_score(y_test, y_pred, average='weighted'), 3)
    result.loc[counter]["F1-Score-Macro"] = round(f1_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["F1-Score-Weighted"] = round(f1_score(y_test, y_pred, average='weighted'), 3)
    counter += 1
  foldcounter += 1
result = result.dropna()
result

Processing Fold 1 ...


  0%|          | 0/29 [00:00<?, ?it/s]

 41%|████▏     | 12/29 [02:41<04:39, 16.44s/it]

In [ ]:
result.to_csv("results/10-Fold-fasttext-all.csv")
result.to_excel("results/10-Fold-fasttext-all.xlsx")

## GLOVE

In [ ]:
data1 = pd.read_csv("datasets/edited/glove_embedded_davidson.csv")
del data1["Unnamed: 0"]
data1

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,0.05,-0.01,-0.02,0.02,0.04,0.00,-0.04,0.01,-0.01,-0.00,...,0.04,-0.05,0.01,0.05,0.04,0.00,0.00,0.07,0.41,0.52
1,-0.04,0.00,0.03,0.01,0.03,0.02,-0.03,-0.02,0.04,0.03,...,-0.02,0.03,-0.01,0.05,0.01,0.03,0.03,0.03,0.43,0.54
2,-0.03,-0.03,0.05,0.02,-0.01,-0.01,-0.00,-0.04,-0.02,0.03,...,-0.02,0.01,0.03,-0.02,-0.03,0.01,0.01,0.00,0.03,0.97
3,0.02,-0.02,0.02,0.02,0.02,-0.08,0.03,-0.04,-0.05,-0.02,...,0.07,-0.09,0.04,-0.04,0.06,0.01,-0.02,0.06,0.73,0.21
4,-0.04,0.00,-0.05,-0.01,0.02,-0.05,0.04,0.01,-0.03,0.02,...,0.02,0.06,0.03,-0.02,-0.03,-0.03,0.03,0.01,0.19,0.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0.03,-0.01,-0.04,0.01,-0.02,-0.06,0.00,-0.01,-0.03,0.00,...,-0.01,-0.02,0.02,0.02,-0.01,0.05,-0.02,0.01,0.12,0.87
24779,0.07,0.06,0.00,-0.06,0.02,0.02,0.03,-0.00,-0.03,0.02,...,-0.00,0.02,-0.06,0.03,0.02,-0.03,-0.02,0.01,0.10,0.89
24780,-0.05,-0.02,0.01,0.03,0.01,0.00,0.01,0.02,0.01,-0.05,...,-0.06,0.01,-0.02,0.01,0.09,-0.03,-0.03,0.04,0.39,0.57
24781,0.05,0.03,0.03,-0.03,-0.08,-0.01,0.02,-0.05,0.05,0.02,...,0.03,-0.02,0.09,-0.01,-0.02,0.01,0.10,0.01,0.11,0.88


In [ ]:
data2 = pd.read_csv("datasets/edited/glove_embedded_mfrc.csv")
del data2["Unnamed: 0"]
data2

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,0.01,0.01,0.01,0.02,-0.01,0.00,0.01,0.02,0.01,0.01,...,-0.01,0.03,0.01,0.01,0.03,0.03,-0.01,0.11,0.32,0.57
1,-0.01,-0.01,-0.02,-0.04,0.02,0.00,-0.01,-0.03,-0.01,-0.03,...,0.03,0.04,0.02,-0.01,0.00,0.01,-0.00,0.17,0.56,0.27
2,-0.02,0.02,0.00,-0.01,-0.05,0.04,-0.02,-0.01,0.00,-0.01,...,-0.02,-0.05,0.02,-0.04,0.05,0.01,-0.03,0.07,0.45,0.49
3,-0.05,0.02,-0.01,0.03,0.01,-0.02,-0.03,-0.04,-0.03,-0.02,...,-0.03,0.05,-0.05,0.00,-0.02,-0.00,-0.01,0.02,0.35,0.63
4,0.01,-0.01,0.00,-0.00,0.01,-0.02,0.01,-0.01,-0.00,-0.03,...,0.00,0.04,0.02,0.01,0.01,-0.01,0.00,0.05,0.78,0.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,-0.01,0.02,0.01,0.01,0.02,-0.01,-0.01,0.02,-0.01,-0.02,...,-0.02,0.02,0.05,-0.01,0.02,0.02,-0.00,0.08,0.17,0.75
17882,0.02,0.01,0.02,0.01,0.02,-0.02,0.00,-0.06,0.02,-0.04,...,0.00,0.03,0.03,-0.02,0.03,0.06,0.02,0.01,0.16,0.83
17883,-0.05,-0.00,0.04,-0.04,0.02,0.02,0.02,0.01,0.01,-0.00,...,0.03,0.04,0.02,0.00,0.00,0.02,-0.00,0.02,0.25,0.73
17884,-0.00,0.02,-0.02,-0.04,-0.03,-0.02,-0.00,0.05,0.00,0.01,...,0.08,0.08,0.10,-0.01,0.05,-0.03,0.06,0.05,0.43,0.52


In [ ]:
data = data1.join(df)
data 

,0,1,2,3,4,5,6,7,8,9,...,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,0.05,-0.01,-0.02,0.02,0.04,0.00,-0.04,0.01,-0.01,-0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,1.00,Partially Negative
1,-0.04,0.00,0.03,0.01,0.03,0.02,-0.03,-0.02,0.04,0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
2,-0.03,-0.03,0.05,0.02,-0.01,-0.01,-0.00,-0.04,-0.02,0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
3,0.02,-0.02,0.02,0.02,0.02,-0.08,0.03,-0.04,-0.05,-0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
4,-0.04,0.00,-0.05,-0.01,0.02,-0.05,0.04,0.01,-0.03,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0.03,-0.01,-0.04,0.01,-0.02,-0.06,0.00,-0.01,-0.03,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.67,0.00,0.33,Morally Positive
24779,0.07,0.06,0.00,-0.06,0.02,0.02,0.03,-0.00,-0.03,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24780,-0.05,-0.02,0.01,0.03,0.01,0.00,0.01,0.02,0.01,-0.05,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24781,0.05,0.03,0.03,-0.03,-0.08,-0.01,0.02,-0.05,0.05,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [ ]:
dataframe2 = data2.join(df2)
dataframe2 

,0,1,2,3,4,5,6,7,8,9,...,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,0.01,0.01,0.01,0.02,-0.01,0.00,0.01,0.02,0.01,0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
1,-0.01,-0.01,-0.02,-0.04,0.02,0.00,-0.01,-0.03,-0.01,-0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
2,-0.02,0.02,0.00,-0.01,-0.05,0.04,-0.02,-0.01,0.00,-0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
3,-0.05,0.02,-0.01,0.03,0.01,-0.02,-0.03,-0.04,-0.03,-0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
4,0.01,-0.01,0.00,-0.00,0.01,-0.02,0.01,-0.01,-0.00,-0.03,...,0.00,0.00,0.00,0.00,0.00,0.33,0.67,0.00,0.00,Morally Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,-0.01,0.02,0.01,0.01,0.02,-0.01,-0.01,0.02,-0.01,-0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
17882,0.02,0.01,0.02,0.01,0.02,-0.02,0.00,-0.06,0.02,-0.04,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
17883,-0.05,-0.00,0.04,-0.04,0.02,0.02,0.02,0.01,0.01,-0.00,...,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,Morally Negative
17884,-0.00,0.02,-0.02,-0.04,-0.03,-0.02,-0.00,0.05,0.00,0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [ ]:
dataframe = pd.concat([data, dataframe2])
dataframe = dataframe.reset_index()
dataframe

,0,1,2,3,4,5,6,7,8,9,...,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,0.05,-0.01,-0.02,0.02,0.04,0.00,-0.04,0.01,-0.01,-0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,1.00,Partially Negative
1,-0.04,0.00,0.03,0.01,0.03,0.02,-0.03,-0.02,0.04,0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
2,-0.03,-0.03,0.05,0.02,-0.01,-0.01,-0.00,-0.04,-0.02,0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
3,0.02,-0.02,0.02,0.02,0.02,-0.08,0.03,-0.04,-0.05,-0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
4,-0.04,0.00,-0.05,-0.01,0.02,-0.05,0.04,0.01,-0.03,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,-0.01,0.02,0.01,0.01,0.02,-0.01,-0.01,0.02,-0.01,-0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
17882,0.02,0.01,0.02,0.01,0.02,-0.02,0.00,-0.06,0.02,-0.04,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
17883,-0.05,-0.00,0.04,-0.04,0.02,0.02,0.02,0.01,0.01,-0.00,...,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,Morally Negative
17884,-0.00,0.02,-0.02,-0.04,-0.03,-0.02,-0.00,0.05,0.00,0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [ ]:
y = dataframe['target']
X = dataframe
del X['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.5,random_state =123)
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

  0%|          | 0/29 [00:00<?, ?it/s]

 97%|█████████▋| 28/29 [01:34<00:03,  3.13s/it]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007504 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26328
[LightGBM] [Info] Number of data points in the train set: 12391, number of used features: 112
[LightGBM] [Info] Start training from score -1.498123
[LightGBM] [Info] Start training from score -2.374603
[LightGBM] [Info] Start training from score -1.542033
[LightGBM] [Info] Start training from score -0.880307
[LightGBM] [Info] Start training from score -3.619591
[LightGBM] [Info] Start training from score -3.744553
[LightGBM] [Info] Start training from score -5.735846
[LightGBM] [Info] Start training from score -6.785668
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

100%|██████████| 29/29 [01:36<00:00,  3.32s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LinearDiscriminantAnalysis,0.98,0.98,None,0.98,0.35
DecisionTreeClassifier,0.99,0.88,None,0.99,2.01
LinearSVC,0.99,0.88,None,0.99,2.34
LogisticRegression,0.99,0.86,None,0.99,0.85
BaggingClassifier,0.99,0.85,None,0.99,12.11
Perceptron,0.98,0.80,None,0.98,0.48
CalibratedClassifierCV,0.99,0.80,None,0.99,10.54
PassiveAggressiveClassifier,0.98,0.79,None,0.98,0.47
NearestCentroid,0.78,0.79,None,0.80,0.17


In [ ]:
result = pd.DataFrame(columns=["Fold","Model","Accuracy","Balanced Accuracy", "Precision-Macro",  "Precision-Weighted", "Recall-Macro", "Recall-Weighted", "F1-Score-Macro", "F1-Score-Weighted"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=42, shuffle=True)

counter = 0
foldcounter = 1
for train_index, test_index in fold.split(X, y):
  print("Processing Fold "+ str(foldcounter) + " ...")
  X_train, X_test, y_train, y_test = \
      X[ X.index.isin(train_index)], X[ X.index.isin(test_index)], y[train_index], y[test_index]
  clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
  models = clf.provide_models(X_train, X_test, y_train, y_test)
  for key, model in models.items():
    y_pred = model.predict(X_test)
    result.loc[counter]["Fold"] = foldcounter
    result.loc[counter]["Model"] = key
    result.loc[counter]["Accuracy"] = round(accuracy_score(y_test, y_pred), 3)
    result.loc[counter]["Balanced Accuracy"] = round(balanced_accuracy_score(y_test, y_pred), 3)
    result.loc[counter]["Precision-Macro"] = round(precision_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["Precision-Weighted"] = round(precision_score(y_test, y_pred, average='weighted'), 3)
    result.loc[counter]["Recall-Macro"] = round(recall_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["Recall-Weighted"] = round(recall_score(y_test, y_pred, average='weighted'), 3)
    result.loc[counter]["F1-Score-Macro"] = round(f1_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["F1-Score-Weighted"] = round(f1_score(y_test, y_pred, average='weighted'), 3)
    counter += 1
  foldcounter += 1
result = result.dropna()
result

Processing Fold 1 ...


 90%|████████▉ | 26/29 [03:01<00:31, 10.37s/it]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010837 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26336
[LightGBM] [Info] Number of data points in the train set: 22304, number of used features: 112
[LightGBM] [Info] Start training from score -1.499336
[LightGBM] [Info] Start training from score -2.387414
[LightGBM] [Info] Start training from score -1.522905
[LightGBM] [Info] Start training from score -0.883174
[LightGBM] [Info] Start training from score -3.666885
[LightGBM] [Info] Start training from score -3.747220
[LightGBM] [Info] Start training from score -5.695033
[LightGBM] [Info] Start training from score -7.122150
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

100%|██████████| 29/29 [03:03<00:00,  6.33s/it]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

...

 90%|████████▉ | 26/29 [03:06<00:32, 10.71s/it]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008602 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26341
[LightGBM] [Info] Number of data points in the train set: 22304, number of used features: 113
[LightGBM] [Info] Start training from score -1.502151
[LightGBM] [Info] Start training from score -2.383517
[LightGBM] [Info] Start training from score -1.529920
[LightGBM] [Info] Start training from score -0.881332
[LightGBM] [Info] Start training from score -3.647771
[LightGBM] [Info] Start training from score -3.735878
[LightGBM] [Info] Start training from score -5.643073
[LightGBM] [Info] Start training from score -7.016789
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

100%|██████████| 29/29 [03:08<00:00,  6.51s/it]


Processing Fold 3 ...


 86%|████████▌ | 25/29 [02:59<00:28,  7.19s/it]


In [ ]:
result.to_csv("results/10-Fold-glove-all.csv")
result.to_excel("results/10-Fold-glove-all.xlsx")

## BERT

In [ ]:
data1 = pd.read_csv("datasets/edited/BERT_embedded_davidson.csv")
del data1["Unnamed: 0"]
data1

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,0.05,-0.01,-0.02,0.02,0.04,0.00,-0.04,0.01,-0.01,-0.00,...,0.04,-0.05,0.01,0.05,0.04,0.00,0.00,0.07,0.41,0.52
1,-0.04,0.00,0.03,0.01,0.03,0.02,-0.03,-0.02,0.04,0.03,...,-0.02,0.03,-0.01,0.05,0.01,0.03,0.03,0.03,0.43,0.54
2,-0.03,-0.03,0.05,0.02,-0.01,-0.01,-0.00,-0.04,-0.02,0.03,...,-0.02,0.01,0.03,-0.02,-0.03,0.01,0.01,0.00,0.03,0.97
3,0.02,-0.02,0.02,0.02,0.02,-0.08,0.03,-0.04,-0.05,-0.02,...,0.07,-0.09,0.04,-0.04,0.06,0.01,-0.02,0.06,0.73,0.21
4,-0.04,0.00,-0.05,-0.01,0.02,-0.05,0.04,0.01,-0.03,0.02,...,0.02,0.06,0.03,-0.02,-0.03,-0.03,0.03,0.01,0.19,0.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0.03,-0.01,-0.04,0.01,-0.02,-0.06,0.00,-0.01,-0.03,0.00,...,-0.01,-0.02,0.02,0.02,-0.01,0.05,-0.02,0.01,0.12,0.87
24779,0.07,0.06,0.00,-0.06,0.02,0.02,0.03,-0.00,-0.03,0.02,...,-0.00,0.02,-0.06,0.03,0.02,-0.03,-0.02,0.01,0.10,0.89
24780,-0.05,-0.02,0.01,0.03,0.01,0.00,0.01,0.02,0.01,-0.05,...,-0.06,0.01,-0.02,0.01,0.09,-0.03,-0.03,0.04,0.39,0.57
24781,0.05,0.03,0.03,-0.03,-0.08,-0.01,0.02,-0.05,0.05,0.02,...,0.03,-0.02,0.09,-0.01,-0.02,0.01,0.10,0.01,0.11,0.88


In [ ]:
data2 = pd.read_csv("datasets/edited/BERT_embedded_mfrc.csv")
del data2["Unnamed: 0"]
data2

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,0.01,0.01,0.01,0.02,-0.01,0.00,0.01,0.02,0.01,0.01,...,-0.01,0.03,0.01,0.01,0.03,0.03,-0.01,0.11,0.32,0.57
1,-0.01,-0.01,-0.02,-0.04,0.02,0.00,-0.01,-0.03,-0.01,-0.03,...,0.03,0.04,0.02,-0.01,0.00,0.01,-0.00,0.17,0.56,0.27
2,-0.02,0.02,0.00,-0.01,-0.05,0.04,-0.02,-0.01,0.00,-0.01,...,-0.02,-0.05,0.02,-0.04,0.05,0.01,-0.03,0.07,0.45,0.49
3,-0.05,0.02,-0.01,0.03,0.01,-0.02,-0.03,-0.04,-0.03,-0.02,...,-0.03,0.05,-0.05,0.00,-0.02,-0.00,-0.01,0.02,0.35,0.63
4,0.01,-0.01,0.00,-0.00,0.01,-0.02,0.01,-0.01,-0.00,-0.03,...,0.00,0.04,0.02,0.01,0.01,-0.01,0.00,0.05,0.78,0.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,-0.01,0.02,0.01,0.01,0.02,-0.01,-0.01,0.02,-0.01,-0.02,...,-0.02,0.02,0.05,-0.01,0.02,0.02,-0.00,0.08,0.17,0.75
17882,0.02,0.01,0.02,0.01,0.02,-0.02,0.00,-0.06,0.02,-0.04,...,0.00,0.03,0.03,-0.02,0.03,0.06,0.02,0.01,0.16,0.83
17883,-0.05,-0.00,0.04,-0.04,0.02,0.02,0.02,0.01,0.01,-0.00,...,0.03,0.04,0.02,0.00,0.00,0.02,-0.00,0.02,0.25,0.73
17884,-0.00,0.02,-0.02,-0.04,-0.03,-0.02,-0.00,0.05,0.00,0.01,...,0.08,0.08,0.10,-0.01,0.05,-0.03,0.06,0.05,0.43,0.52


In [ ]:
data = data1.join(df)
data 

,0,1,2,3,4,5,6,7,8,9,...,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,0.05,-0.01,-0.02,0.02,0.04,0.00,-0.04,0.01,-0.01,-0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,1.00,Partially Negative
1,-0.04,0.00,0.03,0.01,0.03,0.02,-0.03,-0.02,0.04,0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
2,-0.03,-0.03,0.05,0.02,-0.01,-0.01,-0.00,-0.04,-0.02,0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
3,0.02,-0.02,0.02,0.02,0.02,-0.08,0.03,-0.04,-0.05,-0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
4,-0.04,0.00,-0.05,-0.01,0.02,-0.05,0.04,0.01,-0.03,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0.03,-0.01,-0.04,0.01,-0.02,-0.06,0.00,-0.01,-0.03,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.67,0.00,0.33,Morally Positive
24779,0.07,0.06,0.00,-0.06,0.02,0.02,0.03,-0.00,-0.03,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24780,-0.05,-0.02,0.01,0.03,0.01,0.00,0.01,0.02,0.01,-0.05,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24781,0.05,0.03,0.03,-0.03,-0.08,-0.01,0.02,-0.05,0.05,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [ ]:
dataframe2 = data2.join(df2)
dataframe2 

,0,1,2,3,4,5,6,7,8,9,...,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,0.01,0.01,0.01,0.02,-0.01,0.00,0.01,0.02,0.01,0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
1,-0.01,-0.01,-0.02,-0.04,0.02,0.00,-0.01,-0.03,-0.01,-0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
2,-0.02,0.02,0.00,-0.01,-0.05,0.04,-0.02,-0.01,0.00,-0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
3,-0.05,0.02,-0.01,0.03,0.01,-0.02,-0.03,-0.04,-0.03,-0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
4,0.01,-0.01,0.00,-0.00,0.01,-0.02,0.01,-0.01,-0.00,-0.03,...,0.00,0.00,0.00,0.00,0.00,0.33,0.67,0.00,0.00,Morally Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,-0.01,0.02,0.01,0.01,0.02,-0.01,-0.01,0.02,-0.01,-0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
17882,0.02,0.01,0.02,0.01,0.02,-0.02,0.00,-0.06,0.02,-0.04,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
17883,-0.05,-0.00,0.04,-0.04,0.02,0.02,0.02,0.01,0.01,-0.00,...,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,Morally Negative
17884,-0.00,0.02,-0.02,-0.04,-0.03,-0.02,-0.00,0.05,0.00,0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [ ]:
dataframe = pd.concat([data, dataframe2])
dataframe = dataframe.reset_index()
dataframe

,0,1,2,3,4,5,6,7,8,9,...,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,0.05,-0.01,-0.02,0.02,0.04,0.00,-0.04,0.01,-0.01,-0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,1.00,Partially Negative
1,-0.04,0.00,0.03,0.01,0.03,0.02,-0.03,-0.02,0.04,0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
2,-0.03,-0.03,0.05,0.02,-0.01,-0.01,-0.00,-0.04,-0.02,0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
3,0.02,-0.02,0.02,0.02,0.02,-0.08,0.03,-0.04,-0.05,-0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
4,-0.04,0.00,-0.05,-0.01,0.02,-0.05,0.04,0.01,-0.03,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,-0.01,0.02,0.01,0.01,0.02,-0.01,-0.01,0.02,-0.01,-0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
17882,0.02,0.01,0.02,0.01,0.02,-0.02,0.00,-0.06,0.02,-0.04,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
17883,-0.05,-0.00,0.04,-0.04,0.02,0.02,0.02,0.01,0.01,-0.00,...,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,Morally Negative
17884,-0.00,0.02,-0.02,-0.04,-0.03,-0.02,-0.00,0.05,0.00,0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [ ]:
y = dataframe['target']
X = dataframe
del X['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.5,random_state =123)
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

  0%|          | 0/29 [00:00<?, ?it/s]

 97%|█████████▋| 28/29 [01:34<00:03,  3.13s/it]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007504 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26328
[LightGBM] [Info] Number of data points in the train set: 12391, number of used features: 112
[LightGBM] [Info] Start training from score -1.498123
[LightGBM] [Info] Start training from score -2.374603
[LightGBM] [Info] Start training from score -1.542033
[LightGBM] [Info] Start training from score -0.880307
[LightGBM] [Info] Start training from score -3.619591
[LightGBM] [Info] Start training from score -3.744553
[LightGBM] [Info] Start training from score -5.735846
[LightGBM] [Info] Start training from score -6.785668
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

100%|██████████| 29/29 [01:36<00:00,  3.32s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LinearDiscriminantAnalysis,0.98,0.98,None,0.98,0.35
DecisionTreeClassifier,0.99,0.88,None,0.99,2.01
LinearSVC,0.99,0.88,None,0.99,2.34
LogisticRegression,0.99,0.86,None,0.99,0.85
BaggingClassifier,0.99,0.85,None,0.99,12.11
Perceptron,0.98,0.80,None,0.98,0.48
CalibratedClassifierCV,0.99,0.80,None,0.99,10.54
PassiveAggressiveClassifier,0.98,0.79,None,0.98,0.47
NearestCentroid,0.78,0.79,None,0.80,0.17


In [ ]:
result = pd.DataFrame(columns=["Fold","Model","Accuracy","Balanced Accuracy", "Precision-Macro",  "Precision-Weighted", "Recall-Macro", "Recall-Weighted", "F1-Score-Macro", "F1-Score-Weighted"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=42, shuffle=True)

counter = 0
foldcounter = 1
for train_index, test_index in fold.split(X, y):
  print("Processing Fold "+ str(foldcounter) + " ...")
  X_train, X_test, y_train, y_test = \
      X[ X.index.isin(train_index)], X[ X.index.isin(test_index)], y[train_index], y[test_index]
  clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
  models = clf.provide_models(X_train, X_test, y_train, y_test)
  for key, model in models.items():
    y_pred = model.predict(X_test)
    result.loc[counter]["Fold"] = foldcounter
    result.loc[counter]["Model"] = key
    result.loc[counter]["Accuracy"] = round(accuracy_score(y_test, y_pred), 3)
    result.loc[counter]["Balanced Accuracy"] = round(balanced_accuracy_score(y_test, y_pred), 3)
    result.loc[counter]["Precision-Macro"] = round(precision_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["Precision-Weighted"] = round(precision_score(y_test, y_pred, average='weighted'), 3)
    result.loc[counter]["Recall-Macro"] = round(recall_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["Recall-Weighted"] = round(recall_score(y_test, y_pred, average='weighted'), 3)
    result.loc[counter]["F1-Score-Macro"] = round(f1_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["F1-Score-Weighted"] = round(f1_score(y_test, y_pred, average='weighted'), 3)
    counter += 1
  foldcounter += 1
result = result.dropna()
result

Processing Fold 1 ...


 90%|████████▉ | 26/29 [03:01<00:31, 10.37s/it]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010837 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26336
[LightGBM] [Info] Number of data points in the train set: 22304, number of used features: 112
[LightGBM] [Info] Start training from score -1.499336
[LightGBM] [Info] Start training from score -2.387414
[LightGBM] [Info] Start training from score -1.522905
[LightGBM] [Info] Start training from score -0.883174
[LightGBM] [Info] Start training from score -3.666885
[LightGBM] [Info] Start training from score -3.747220
[LightGBM] [Info] Start training from score -5.695033
[LightGBM] [Info] Start training from score -7.122150
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

100%|██████████| 29/29 [03:03<00:00,  6.33s/it]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

...

 90%|████████▉ | 26/29 [03:06<00:32, 10.71s/it]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008602 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26341
[LightGBM] [Info] Number of data points in the train set: 22304, number of used features: 113
[LightGBM] [Info] Start training from score -1.502151
[LightGBM] [Info] Start training from score -2.383517
[LightGBM] [Info] Start training from score -1.529920
[LightGBM] [Info] Start training from score -0.881332
[LightGBM] [Info] Start training from score -3.647771
[LightGBM] [Info] Start training from score -3.735878
[LightGBM] [Info] Start training from score -5.643073
[LightGBM] [Info] Start training from score -7.016789
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

100%|██████████| 29/29 [03:08<00:00,  6.51s/it]


Processing Fold 3 ...


 86%|████████▌ | 25/29 [02:59<00:28,  7.19s/it]


In [ ]:
result.to_csv("results/10-Fold-BERT-all.csv")
result.to_excel("results/10-Fold-BERT-all.xlsx")

## WORD2VEC

In [ ]:
data1 = pd.read_csv("datasets/edited/word2vec_embedded_davidson.csv")
del data1["Unnamed: 0"]
data1

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,0.05,-0.01,-0.02,0.02,0.04,0.00,-0.04,0.01,-0.01,-0.00,...,0.04,-0.05,0.01,0.05,0.04,0.00,0.00,0.07,0.41,0.52
1,-0.04,0.00,0.03,0.01,0.03,0.02,-0.03,-0.02,0.04,0.03,...,-0.02,0.03,-0.01,0.05,0.01,0.03,0.03,0.03,0.43,0.54
2,-0.03,-0.03,0.05,0.02,-0.01,-0.01,-0.00,-0.04,-0.02,0.03,...,-0.02,0.01,0.03,-0.02,-0.03,0.01,0.01,0.00,0.03,0.97
3,0.02,-0.02,0.02,0.02,0.02,-0.08,0.03,-0.04,-0.05,-0.02,...,0.07,-0.09,0.04,-0.04,0.06,0.01,-0.02,0.06,0.73,0.21
4,-0.04,0.00,-0.05,-0.01,0.02,-0.05,0.04,0.01,-0.03,0.02,...,0.02,0.06,0.03,-0.02,-0.03,-0.03,0.03,0.01,0.19,0.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0.03,-0.01,-0.04,0.01,-0.02,-0.06,0.00,-0.01,-0.03,0.00,...,-0.01,-0.02,0.02,0.02,-0.01,0.05,-0.02,0.01,0.12,0.87
24779,0.07,0.06,0.00,-0.06,0.02,0.02,0.03,-0.00,-0.03,0.02,...,-0.00,0.02,-0.06,0.03,0.02,-0.03,-0.02,0.01,0.10,0.89
24780,-0.05,-0.02,0.01,0.03,0.01,0.00,0.01,0.02,0.01,-0.05,...,-0.06,0.01,-0.02,0.01,0.09,-0.03,-0.03,0.04,0.39,0.57
24781,0.05,0.03,0.03,-0.03,-0.08,-0.01,0.02,-0.05,0.05,0.02,...,0.03,-0.02,0.09,-0.01,-0.02,0.01,0.10,0.01,0.11,0.88


In [ ]:
data2 = pd.read_csv("datasets/edited/word2vec_embedded_mfrc.csv")
del data2["Unnamed: 0"]
data2

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,0.01,0.01,0.01,0.02,-0.01,0.00,0.01,0.02,0.01,0.01,...,-0.01,0.03,0.01,0.01,0.03,0.03,-0.01,0.11,0.32,0.57
1,-0.01,-0.01,-0.02,-0.04,0.02,0.00,-0.01,-0.03,-0.01,-0.03,...,0.03,0.04,0.02,-0.01,0.00,0.01,-0.00,0.17,0.56,0.27
2,-0.02,0.02,0.00,-0.01,-0.05,0.04,-0.02,-0.01,0.00,-0.01,...,-0.02,-0.05,0.02,-0.04,0.05,0.01,-0.03,0.07,0.45,0.49
3,-0.05,0.02,-0.01,0.03,0.01,-0.02,-0.03,-0.04,-0.03,-0.02,...,-0.03,0.05,-0.05,0.00,-0.02,-0.00,-0.01,0.02,0.35,0.63
4,0.01,-0.01,0.00,-0.00,0.01,-0.02,0.01,-0.01,-0.00,-0.03,...,0.00,0.04,0.02,0.01,0.01,-0.01,0.00,0.05,0.78,0.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,-0.01,0.02,0.01,0.01,0.02,-0.01,-0.01,0.02,-0.01,-0.02,...,-0.02,0.02,0.05,-0.01,0.02,0.02,-0.00,0.08,0.17,0.75
17882,0.02,0.01,0.02,0.01,0.02,-0.02,0.00,-0.06,0.02,-0.04,...,0.00,0.03,0.03,-0.02,0.03,0.06,0.02,0.01,0.16,0.83
17883,-0.05,-0.00,0.04,-0.04,0.02,0.02,0.02,0.01,0.01,-0.00,...,0.03,0.04,0.02,0.00,0.00,0.02,-0.00,0.02,0.25,0.73
17884,-0.00,0.02,-0.02,-0.04,-0.03,-0.02,-0.00,0.05,0.00,0.01,...,0.08,0.08,0.10,-0.01,0.05,-0.03,0.06,0.05,0.43,0.52


In [ ]:
data = data1.join(df)
data 

,0,1,2,3,4,5,6,7,8,9,...,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,0.05,-0.01,-0.02,0.02,0.04,0.00,-0.04,0.01,-0.01,-0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,1.00,Partially Negative
1,-0.04,0.00,0.03,0.01,0.03,0.02,-0.03,-0.02,0.04,0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
2,-0.03,-0.03,0.05,0.02,-0.01,-0.01,-0.00,-0.04,-0.02,0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
3,0.02,-0.02,0.02,0.02,0.02,-0.08,0.03,-0.04,-0.05,-0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
4,-0.04,0.00,-0.05,-0.01,0.02,-0.05,0.04,0.01,-0.03,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0.03,-0.01,-0.04,0.01,-0.02,-0.06,0.00,-0.01,-0.03,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.67,0.00,0.33,Morally Positive
24779,0.07,0.06,0.00,-0.06,0.02,0.02,0.03,-0.00,-0.03,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24780,-0.05,-0.02,0.01,0.03,0.01,0.00,0.01,0.02,0.01,-0.05,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24781,0.05,0.03,0.03,-0.03,-0.08,-0.01,0.02,-0.05,0.05,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [ ]:
dataframe2 = data2.join(df2)
dataframe2 

,0,1,2,3,4,5,6,7,8,9,...,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,0.01,0.01,0.01,0.02,-0.01,0.00,0.01,0.02,0.01,0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
1,-0.01,-0.01,-0.02,-0.04,0.02,0.00,-0.01,-0.03,-0.01,-0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
2,-0.02,0.02,0.00,-0.01,-0.05,0.04,-0.02,-0.01,0.00,-0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
3,-0.05,0.02,-0.01,0.03,0.01,-0.02,-0.03,-0.04,-0.03,-0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
4,0.01,-0.01,0.00,-0.00,0.01,-0.02,0.01,-0.01,-0.00,-0.03,...,0.00,0.00,0.00,0.00,0.00,0.33,0.67,0.00,0.00,Morally Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,-0.01,0.02,0.01,0.01,0.02,-0.01,-0.01,0.02,-0.01,-0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
17882,0.02,0.01,0.02,0.01,0.02,-0.02,0.00,-0.06,0.02,-0.04,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
17883,-0.05,-0.00,0.04,-0.04,0.02,0.02,0.02,0.01,0.01,-0.00,...,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,Morally Negative
17884,-0.00,0.02,-0.02,-0.04,-0.03,-0.02,-0.00,0.05,0.00,0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [ ]:
dataframe = pd.concat([data, dataframe2])
dataframe = dataframe.reset_index()
dataframe

,0,1,2,3,4,5,6,7,8,9,...,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,0.05,-0.01,-0.02,0.02,0.04,0.00,-0.04,0.01,-0.01,-0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,1.00,Partially Negative
1,-0.04,0.00,0.03,0.01,0.03,0.02,-0.03,-0.02,0.04,0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
2,-0.03,-0.03,0.05,0.02,-0.01,-0.01,-0.00,-0.04,-0.02,0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
3,0.02,-0.02,0.02,0.02,0.02,-0.08,0.03,-0.04,-0.05,-0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
4,-0.04,0.00,-0.05,-0.01,0.02,-0.05,0.04,0.01,-0.03,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,-0.01,0.02,0.01,0.01,0.02,-0.01,-0.01,0.02,-0.01,-0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
17882,0.02,0.01,0.02,0.01,0.02,-0.02,0.00,-0.06,0.02,-0.04,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
17883,-0.05,-0.00,0.04,-0.04,0.02,0.02,0.02,0.01,0.01,-0.00,...,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,Morally Negative
17884,-0.00,0.02,-0.02,-0.04,-0.03,-0.02,-0.00,0.05,0.00,0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [ ]:
y = dataframe['target']
X = dataframe
del X['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.5,random_state =123)
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

  0%|          | 0/29 [00:00<?, ?it/s]

 97%|█████████▋| 28/29 [01:34<00:03,  3.13s/it]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007504 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26328
[LightGBM] [Info] Number of data points in the train set: 12391, number of used features: 112
[LightGBM] [Info] Start training from score -1.498123
[LightGBM] [Info] Start training from score -2.374603
[LightGBM] [Info] Start training from score -1.542033
[LightGBM] [Info] Start training from score -0.880307
[LightGBM] [Info] Start training from score -3.619591
[LightGBM] [Info] Start training from score -3.744553
[LightGBM] [Info] Start training from score -5.735846
[LightGBM] [Info] Start training from score -6.785668
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

100%|██████████| 29/29 [01:36<00:00,  3.32s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LinearDiscriminantAnalysis,0.98,0.98,None,0.98,0.35
DecisionTreeClassifier,0.99,0.88,None,0.99,2.01
LinearSVC,0.99,0.88,None,0.99,2.34
LogisticRegression,0.99,0.86,None,0.99,0.85
BaggingClassifier,0.99,0.85,None,0.99,12.11
Perceptron,0.98,0.80,None,0.98,0.48
CalibratedClassifierCV,0.99,0.80,None,0.99,10.54
PassiveAggressiveClassifier,0.98,0.79,None,0.98,0.47
NearestCentroid,0.78,0.79,None,0.80,0.17


In [ ]:
result = pd.DataFrame(columns=["Fold","Model","Accuracy","Balanced Accuracy", "Precision-Macro",  "Precision-Weighted", "Recall-Macro", "Recall-Weighted", "F1-Score-Macro", "F1-Score-Weighted"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=42, shuffle=True)

counter = 0
foldcounter = 1
for train_index, test_index in fold.split(X, y):
  print("Processing Fold "+ str(foldcounter) + " ...")
  X_train, X_test, y_train, y_test = \
      X[ X.index.isin(train_index)], X[ X.index.isin(test_index)], y[train_index], y[test_index]
  clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
  models = clf.provide_models(X_train, X_test, y_train, y_test)
  for key, model in models.items():
    y_pred = model.predict(X_test)
    result.loc[counter]["Fold"] = foldcounter
    result.loc[counter]["Model"] = key
    result.loc[counter]["Accuracy"] = round(accuracy_score(y_test, y_pred), 3)
    result.loc[counter]["Balanced Accuracy"] = round(balanced_accuracy_score(y_test, y_pred), 3)
    result.loc[counter]["Precision-Macro"] = round(precision_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["Precision-Weighted"] = round(precision_score(y_test, y_pred, average='weighted'), 3)
    result.loc[counter]["Recall-Macro"] = round(recall_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["Recall-Weighted"] = round(recall_score(y_test, y_pred, average='weighted'), 3)
    result.loc[counter]["F1-Score-Macro"] = round(f1_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["F1-Score-Weighted"] = round(f1_score(y_test, y_pred, average='weighted'), 3)
    counter += 1
  foldcounter += 1
result = result.dropna()
result

Processing Fold 1 ...


 90%|████████▉ | 26/29 [03:01<00:31, 10.37s/it]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010837 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26336
[LightGBM] [Info] Number of data points in the train set: 22304, number of used features: 112
[LightGBM] [Info] Start training from score -1.499336
[LightGBM] [Info] Start training from score -2.387414
[LightGBM] [Info] Start training from score -1.522905
[LightGBM] [Info] Start training from score -0.883174
[LightGBM] [Info] Start training from score -3.666885
[LightGBM] [Info] Start training from score -3.747220
[LightGBM] [Info] Start training from score -5.695033
[LightGBM] [Info] Start training from score -7.122150
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

100%|██████████| 29/29 [03:03<00:00,  6.33s/it]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

...

 90%|████████▉ | 26/29 [03:06<00:32, 10.71s/it]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008602 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26341
[LightGBM] [Info] Number of data points in the train set: 22304, number of used features: 113
[LightGBM] [Info] Start training from score -1.502151
[LightGBM] [Info] Start training from score -2.383517
[LightGBM] [Info] Start training from score -1.529920
[LightGBM] [Info] Start training from score -0.881332
[LightGBM] [Info] Start training from score -3.647771
[LightGBM] [Info] Start training from score -3.735878
[LightGBM] [Info] Start training from score -5.643073
[LightGBM] [Info] Start training from score -7.016789
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

100%|██████████| 29/29 [03:08<00:00,  6.51s/it]


Processing Fold 3 ...


 86%|████████▌ | 25/29 [02:59<00:28,  7.19s/it]


In [ ]:
result.to_csv("results/10-Fold-word2vec-all.csv")
result.to_excel("results/10-Fold-word2vec-all.xlsx")

## TFDIF

In [ ]:
data1 = pd.read_csv("datasets/edited/tfdif_embedded_davidson.csv")
del data1["Unnamed: 0"]
data1

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,0.05,-0.01,-0.02,0.02,0.04,0.00,-0.04,0.01,-0.01,-0.00,...,0.04,-0.05,0.01,0.05,0.04,0.00,0.00,0.07,0.41,0.52
1,-0.04,0.00,0.03,0.01,0.03,0.02,-0.03,-0.02,0.04,0.03,...,-0.02,0.03,-0.01,0.05,0.01,0.03,0.03,0.03,0.43,0.54
2,-0.03,-0.03,0.05,0.02,-0.01,-0.01,-0.00,-0.04,-0.02,0.03,...,-0.02,0.01,0.03,-0.02,-0.03,0.01,0.01,0.00,0.03,0.97
3,0.02,-0.02,0.02,0.02,0.02,-0.08,0.03,-0.04,-0.05,-0.02,...,0.07,-0.09,0.04,-0.04,0.06,0.01,-0.02,0.06,0.73,0.21
4,-0.04,0.00,-0.05,-0.01,0.02,-0.05,0.04,0.01,-0.03,0.02,...,0.02,0.06,0.03,-0.02,-0.03,-0.03,0.03,0.01,0.19,0.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0.03,-0.01,-0.04,0.01,-0.02,-0.06,0.00,-0.01,-0.03,0.00,...,-0.01,-0.02,0.02,0.02,-0.01,0.05,-0.02,0.01,0.12,0.87
24779,0.07,0.06,0.00,-0.06,0.02,0.02,0.03,-0.00,-0.03,0.02,...,-0.00,0.02,-0.06,0.03,0.02,-0.03,-0.02,0.01,0.10,0.89
24780,-0.05,-0.02,0.01,0.03,0.01,0.00,0.01,0.02,0.01,-0.05,...,-0.06,0.01,-0.02,0.01,0.09,-0.03,-0.03,0.04,0.39,0.57
24781,0.05,0.03,0.03,-0.03,-0.08,-0.01,0.02,-0.05,0.05,0.02,...,0.03,-0.02,0.09,-0.01,-0.02,0.01,0.10,0.01,0.11,0.88


In [ ]:
data2 = pd.read_csv("datasets/edited/tfdif_embedded_mfrc.csv")
del data2["Unnamed: 0"]
data2

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,0.01,0.01,0.01,0.02,-0.01,0.00,0.01,0.02,0.01,0.01,...,-0.01,0.03,0.01,0.01,0.03,0.03,-0.01,0.11,0.32,0.57
1,-0.01,-0.01,-0.02,-0.04,0.02,0.00,-0.01,-0.03,-0.01,-0.03,...,0.03,0.04,0.02,-0.01,0.00,0.01,-0.00,0.17,0.56,0.27
2,-0.02,0.02,0.00,-0.01,-0.05,0.04,-0.02,-0.01,0.00,-0.01,...,-0.02,-0.05,0.02,-0.04,0.05,0.01,-0.03,0.07,0.45,0.49
3,-0.05,0.02,-0.01,0.03,0.01,-0.02,-0.03,-0.04,-0.03,-0.02,...,-0.03,0.05,-0.05,0.00,-0.02,-0.00,-0.01,0.02,0.35,0.63
4,0.01,-0.01,0.00,-0.00,0.01,-0.02,0.01,-0.01,-0.00,-0.03,...,0.00,0.04,0.02,0.01,0.01,-0.01,0.00,0.05,0.78,0.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,-0.01,0.02,0.01,0.01,0.02,-0.01,-0.01,0.02,-0.01,-0.02,...,-0.02,0.02,0.05,-0.01,0.02,0.02,-0.00,0.08,0.17,0.75
17882,0.02,0.01,0.02,0.01,0.02,-0.02,0.00,-0.06,0.02,-0.04,...,0.00,0.03,0.03,-0.02,0.03,0.06,0.02,0.01,0.16,0.83
17883,-0.05,-0.00,0.04,-0.04,0.02,0.02,0.02,0.01,0.01,-0.00,...,0.03,0.04,0.02,0.00,0.00,0.02,-0.00,0.02,0.25,0.73
17884,-0.00,0.02,-0.02,-0.04,-0.03,-0.02,-0.00,0.05,0.00,0.01,...,0.08,0.08,0.10,-0.01,0.05,-0.03,0.06,0.05,0.43,0.52


In [ ]:
data = data1.join(df)
data 

,0,1,2,3,4,5,6,7,8,9,...,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,0.05,-0.01,-0.02,0.02,0.04,0.00,-0.04,0.01,-0.01,-0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,1.00,Partially Negative
1,-0.04,0.00,0.03,0.01,0.03,0.02,-0.03,-0.02,0.04,0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
2,-0.03,-0.03,0.05,0.02,-0.01,-0.01,-0.00,-0.04,-0.02,0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
3,0.02,-0.02,0.02,0.02,0.02,-0.08,0.03,-0.04,-0.05,-0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
4,-0.04,0.00,-0.05,-0.01,0.02,-0.05,0.04,0.01,-0.03,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0.03,-0.01,-0.04,0.01,-0.02,-0.06,0.00,-0.01,-0.03,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.67,0.00,0.33,Morally Positive
24779,0.07,0.06,0.00,-0.06,0.02,0.02,0.03,-0.00,-0.03,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24780,-0.05,-0.02,0.01,0.03,0.01,0.00,0.01,0.02,0.01,-0.05,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
24781,0.05,0.03,0.03,-0.03,-0.08,-0.01,0.02,-0.05,0.05,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [ ]:
dataframe2 = data2.join(df2)
dataframe2 

,0,1,2,3,4,5,6,7,8,9,...,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,0.01,0.01,0.01,0.02,-0.01,0.00,0.01,0.02,0.01,0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
1,-0.01,-0.01,-0.02,-0.04,0.02,0.00,-0.01,-0.03,-0.01,-0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
2,-0.02,0.02,0.00,-0.01,-0.05,0.04,-0.02,-0.01,0.00,-0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
3,-0.05,0.02,-0.01,0.03,0.01,-0.02,-0.03,-0.04,-0.03,-0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
4,0.01,-0.01,0.00,-0.00,0.01,-0.02,0.01,-0.01,-0.00,-0.03,...,0.00,0.00,0.00,0.00,0.00,0.33,0.67,0.00,0.00,Morally Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,-0.01,0.02,0.01,0.01,0.02,-0.01,-0.01,0.02,-0.01,-0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
17882,0.02,0.01,0.02,0.01,0.02,-0.02,0.00,-0.06,0.02,-0.04,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
17883,-0.05,-0.00,0.04,-0.04,0.02,0.02,0.02,0.01,0.01,-0.00,...,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,Morally Negative
17884,-0.00,0.02,-0.02,-0.04,-0.03,-0.02,-0.00,0.05,0.00,0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [ ]:
dataframe = pd.concat([data, dataframe2])
dataframe = dataframe.reset_index()
dataframe

,0,1,2,3,4,5,6,7,8,9,...,care.vice,authority.virtue,fairness.vice,fairness.virtue,loyalty.vice,loyalty.virtue,sanctity.virtue,authority.vice,sanctity.vice,target
0,0.05,-0.01,-0.02,0.02,0.04,0.00,-0.04,0.01,-0.01,-0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,1.00,Partially Negative
1,-0.04,0.00,0.03,0.01,0.03,0.02,-0.03,-0.02,0.04,0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
2,-0.03,-0.03,0.05,0.02,-0.01,-0.01,-0.00,-0.04,-0.02,0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
3,0.02,-0.02,0.02,0.02,0.02,-0.08,0.03,-0.04,-0.05,-0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral
4,-0.04,0.00,-0.05,-0.01,0.02,-0.05,0.04,0.01,-0.03,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,-0.01,0.02,0.01,0.01,0.02,-0.01,-0.01,0.02,-0.01,-0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment
17882,0.02,0.01,0.02,0.01,0.02,-0.02,0.00,-0.06,0.02,-0.04,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,Morally Negative
17883,-0.05,-0.00,0.04,-0.04,0.02,0.02,0.02,0.01,0.01,-0.00,...,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,Morally Negative
17884,-0.00,0.02,-0.02,-0.04,-0.03,-0.02,-0.00,0.05,0.00,0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Neutral but Negative Sentiment


In [ ]:
y = dataframe['target']
X = dataframe
del X['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.5,random_state =123)
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

  0%|          | 0/29 [00:00<?, ?it/s]

 97%|█████████▋| 28/29 [01:34<00:03,  3.13s/it]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007504 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26328
[LightGBM] [Info] Number of data points in the train set: 12391, number of used features: 112
[LightGBM] [Info] Start training from score -1.498123
[LightGBM] [Info] Start training from score -2.374603
[LightGBM] [Info] Start training from score -1.542033
[LightGBM] [Info] Start training from score -0.880307
[LightGBM] [Info] Start training from score -3.619591
[LightGBM] [Info] Start training from score -3.744553
[LightGBM] [Info] Start training from score -5.735846
[LightGBM] [Info] Start training from score -6.785668
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

100%|██████████| 29/29 [01:36<00:00,  3.32s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LinearDiscriminantAnalysis,0.98,0.98,None,0.98,0.35
DecisionTreeClassifier,0.99,0.88,None,0.99,2.01
LinearSVC,0.99,0.88,None,0.99,2.34
LogisticRegression,0.99,0.86,None,0.99,0.85
BaggingClassifier,0.99,0.85,None,0.99,12.11
Perceptron,0.98,0.80,None,0.98,0.48
CalibratedClassifierCV,0.99,0.80,None,0.99,10.54
PassiveAggressiveClassifier,0.98,0.79,None,0.98,0.47
NearestCentroid,0.78,0.79,None,0.80,0.17


In [ ]:
result = pd.DataFrame(columns=["Fold","Model","Accuracy","Balanced Accuracy", "Precision-Macro",  "Precision-Weighted", "Recall-Macro", "Recall-Weighted", "F1-Score-Macro", "F1-Score-Weighted"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=42, shuffle=True)

counter = 0
foldcounter = 1
for train_index, test_index in fold.split(X, y):
  print("Processing Fold "+ str(foldcounter) + " ...")
  X_train, X_test, y_train, y_test = \
      X[ X.index.isin(train_index)], X[ X.index.isin(test_index)], y[train_index], y[test_index]
  clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
  models = clf.provide_models(X_train, X_test, y_train, y_test)
  for key, model in models.items():
    y_pred = model.predict(X_test)
    result.loc[counter]["Fold"] = foldcounter
    result.loc[counter]["Model"] = key
    result.loc[counter]["Accuracy"] = round(accuracy_score(y_test, y_pred), 3)
    result.loc[counter]["Balanced Accuracy"] = round(balanced_accuracy_score(y_test, y_pred), 3)
    result.loc[counter]["Precision-Macro"] = round(precision_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["Precision-Weighted"] = round(precision_score(y_test, y_pred, average='weighted'), 3)
    result.loc[counter]["Recall-Macro"] = round(recall_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["Recall-Weighted"] = round(recall_score(y_test, y_pred, average='weighted'), 3)
    result.loc[counter]["F1-Score-Macro"] = round(f1_score(y_test, y_pred, average='macro'), 3)
    result.loc[counter]["F1-Score-Weighted"] = round(f1_score(y_test, y_pred, average='weighted'), 3)
    counter += 1
  foldcounter += 1
result = result.dropna()
result

Processing Fold 1 ...


 90%|████████▉ | 26/29 [03:01<00:31, 10.37s/it]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010837 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26336
[LightGBM] [Info] Number of data points in the train set: 22304, number of used features: 112
[LightGBM] [Info] Start training from score -1.499336
[LightGBM] [Info] Start training from score -2.387414
[LightGBM] [Info] Start training from score -1.522905
[LightGBM] [Info] Start training from score -0.883174
[LightGBM] [Info] Start training from score -3.666885
[LightGBM] [Info] Start training from score -3.747220
[LightGBM] [Info] Start training from score -5.695033
[LightGBM] [Info] Start training from score -7.122150
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

100%|██████████| 29/29 [03:03<00:00,  6.33s/it]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

...

 90%|████████▉ | 26/29 [03:06<00:32, 10.71s/it]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008602 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26341
[LightGBM] [Info] Number of data points in the train set: 22304, number of used features: 113
[LightGBM] [Info] Start training from score -1.502151
[LightGBM] [Info] Start training from score -2.383517
[LightGBM] [Info] Start training from score -1.529920
[LightGBM] [Info] Start training from score -0.881332
[LightGBM] [Info] Start training from score -3.647771
[LightGBM] [Info] Start training from score -3.735878
[LightGBM] [Info] Start training from score -5.643073
[LightGBM] [Info] Start training from score -7.016789
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

100%|██████████| 29/29 [03:08<00:00,  6.51s/it]


Processing Fold 3 ...


 86%|████████▌ | 25/29 [02:59<00:28,  7.19s/it]


In [ ]:
result.to_csv("results/10-Fold-tfdif-all.csv")
result.to_excel("results/10-Fold-tfdif-all.xlsx")